# Importación de librerías
=============================================================================================================

In [1]:
import pandas as pd
import numpy as np
import os

import json
import glob
import gzip
import pickle

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from datetime import datetime

import pyarrow.parquet as pq
import pyarrow as pa
from google.cloud import storage

import math


In [2]:
# Se eliminan advertencias
import warnings

# Suprimir las advertencias SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

In [3]:
# Se crea una función para subir los dataframes preprocesados a Google Cloud Storage

# Configura tus credenciales de Google Cloud
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'path/to/your/credentials.json'

# Función que sube el archivo a Google Cloud Storage
def upload_to_gcs(bucket_name, source_file_name, destination_blob_name):
    """Sube un archivo a un bucket de Google Cloud Storage."""
    # Inicializa el cliente de GCS
    storage_client = storage.Client()
    # Obtén el bucket
    bucket = storage_client.bucket(bucket_name)
    # Crea un blob en el bucket
    blob = bucket.blob(destination_blob_name)
    # Sube el archivo al blob
    blob.upload_from_filename(source_file_name)
    print(f"Archivo {source_file_name} subido a {destination_blob_name}.")

# Función que guarda el dataframe en un archivo temporal de parquet y lo sube
def dataframe_to_parquet_and_upload(df, bucket_name, destination_blob_name):
    # Guarda el DataFrame como archivo Parquet
    table = pa.Table.from_pandas(df)
    pq.write_table(table, 'temp.parquet')

    # Sube el archivo Parquet a Google Cloud Storage
    upload_to_gcs(bucket_name, 'temp.parquet', destination_blob_name)

    # Elimina el archivo temporal
    os.remove('temp.parquet')

In [4]:
# Nombre del bucket de GCS
bucket_name = 'archivos-preprocesados-henry'

# ETL de Google - 1

## Carga de datos
=============================================================================================================

In [3]:
# Ruta a la ubicación de los archivos JSON
ruta_directorio = '../Data/Google/metadata-sitios'

dataframes = []
for filename in os.listdir(ruta_directorio):        
    if filename.endswith('.json'):
        df_temporal = pd.read_json(os.path.join(ruta_directorio, filename), lines=True)
        dataframes.append(df_temporal)

dfg_sites = pd.concat(dataframes, ignore_index=True)        

In [4]:
print(dfg_sites.info())
dfg_sites.sample(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3025011 entries, 0 to 3025010
Data columns (total 15 columns):
 #   Column            Dtype  
---  ------            -----  
 0   name              object 
 1   address           object 
 2   gmap_id           object 
 3   description       object 
 4   latitude          float64
 5   longitude         float64
 6   category          object 
 7   avg_rating        float64
 8   num_of_reviews    int64  
 9   price             object 
 10  hours             object 
 11  MISC              object 
 12  state             object 
 13  relative_results  object 
 14  url               object 
dtypes: float64(3), int64(1), object(11)
memory usage: 346.2+ MB
None


,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
1292896,Road Star Auto Sales,"Road Star Auto Sales, 14910 Meridian Ave E, Pu...",0x5490fda53aad9069:0x14e3e4fc44c0672c,None,47.120693,-122.293661,[Car dealer],4.7,45,None,"[[Monday, 10AM–6PM], [Tuesday, 10AM–6PM], [Wed...",None,Closed ⋅ Opens 10AM Tue,"[0x5490fd3c79d2ea5b:0x8bf353cb4df64a65, 0x5490...",https://www.google.com/maps/place//data=!4m2!3...
738208,Dry Clean Best,"Dry Clean Best, 605 E El Camino Real # 2, Sunn...",0x808fb670d017dcdd:0x7e179ae4e11a40ef,None,37.361912,-122.024475,[Dry cleaner],4.4,8,None,"[[Tuesday, 7AM–6PM], [Wednesday, 7AM–6PM], [Th...",{'Accessibility': ['Wheelchair accessible entr...,Closed ⋅ Opens 7AM,"[0x808fb661537793d7:0xa24aeb299ba4ac98, 0x808f...",https://www.google.com/maps/place//data=!4m2!3...


## Exportación de datos
=========================================================

In [5]:
# dfg_sites.to_csv('dfg_sites.csv', index=False)
dfg_sites.to_parquet('../Data/Google/reviews-estados-parquet/dfg_sites.parquet')

## Obtención de datos de los restaurantes
=========================================================

In [6]:
# Carga del dataframe de Sites
dfg_sites = pd.read_parquet('../Data/Google/reviews-estados-parquet/dfg_sites.parquet')

In [7]:
# Se obtiene la categoría de los negocios.
negocios_cat = dfg_sites[["gmap_id","category"]].dropna()
# Se separan las categorías de los negocios
negocios_cat = negocios_cat.explode("category")
negocios_cat.head()

,gmap_id,category
0,0x88f16e41928ff687:0x883dad4fd048e8f8,Pharmacy
1,0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,Textile exporter
2,0x80c2c778e3b73d33:0xbdc58662a4a97d49,Korean restaurant
3,0x80c2c89923b27a41:0x32041559418d447,Fabric store
4,0x80c2c632f933b073:0xc31785961fe826a6,Fabric store


In [8]:
# Se observa la distribución de las categorías.
negocios_cat["category"].value_counts(normalize=True).head(15)

category
Restaurant               0.015530
Auto repair shop         0.012431
Gas station              0.011200
Service establishment    0.010615
Beauty salon             0.010493
Convenience store        0.009926
Church                   0.008474
Hair salon               0.008266
Nail salon               0.007121
ATM                      0.006779
Park                     0.006772
Doctor                   0.005855
Clothing store           0.005823
Dentist                  0.005770
Insurance agency         0.005697
Name: proportion, dtype: float64

In [9]:
# Se obtienen los id de los negocios cuyas categorías incluyan restaurantes o bares.
id_rest = negocios_cat[negocios_cat["category"].str.contains('restaurant|bar', case=False)]
id_rest.head(10)

,gmap_id,category
2,0x80c2c778e3b73d33:0xbdc58662a4a97d49,Korean restaurant
6,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,Restaurant
8,0x87ec235c54d25b31:0x3b75fb5facc602f,Health food restaurant
68,0x89c6c89efcaed69d:0xded973f6033e7dba,Restaurant
75,0x87fd0e70c5f5d87b:0xdf340eeb75040ef3,Bar
87,0x88e711bd2244fe3b:0x8406dd780f0574d1,Barber shop
96,0x7c00456eecad3111:0x8217f9600c51f33,Restaurant
111,0x89a911d03136ef51:0xb1c06825101afe51,Barber shop
114,0x88db4147b1d9e6f3:0x943dbd10a92ba1b1,Restaurant
115,0x88c2e4e34f1ed783:0x76c5da381c499d79,Buffet restaurant


In [10]:
# Se observan los elementos que contienen "bar" dentro de su categoría para identificar aquellos que no corresponden o pertenecen a la categoría de negocios que se espera.
id_rest[id_rest["category"].str.contains("bar",case=False)]["category"].unique()

array(['Bar', 'Barber shop', 'Cocktail bar', 'Bar & grill', 'Sports bar',
       'Wine bar', 'Bark supplier', 'Barbecue restaurant', 'Snack bar',
       'Barber supply store', 'Gay bar', 'Espresso bar', 'Eyebrow bar',
       'Hookah bar', 'Karaoke bar', 'Barn', 'Bartending school',
       'Oyster bar restaurant', 'Poke bar', 'Tapas bar', 'Barber school',
       'Piano bar', 'Subaru dealer', 'Barrister', 'Bar stool supplier',
       'Dart bar', 'Live music bar', 'Korean barbecue restaurant',
       'Cider bar', 'Mongolian barbecue restaurant', 'Cabaret club',
       'Barbecue area', 'Hyperbaric medicine physician',
       'Bar restaurant furniture store', 'Barrel supplier',
       'Bariatric surgeon', 'Bar PMU', 'Salsa bar', 'Military barracks',
       'Tiki bar', 'Stand bar', 'Bar tabac', 'Army barracks',
       'Offal barbecue restaurant', 'Cured ham bar', 'Girl bar',
       'Mutton barbecue restaurant'], dtype=object)

In [11]:
# Se crea una lista con los elementos que no pertenecen a la categoría de negocios esperada.
no = ['Barber shop', 'Bark supplier',
       'Barber supply store', 'Eyebrow bar',
       'Hookah bar', 'Bartending school',
       'Barber school', 'Subaru dealer', 'Barrister', 'Bar stool supplier',
       'Cabaret club', 'Barbecue area', 'Hyperbaric medicine physician',
       'Bar restaurant furniture store', 'Barrel supplier',
       'Bariatric surgeon', 'Military barracks', 'Army barracks']

In [12]:
# Se filtra nuevamente el dataset de las categorías para obtener únicamente las categorías de los negocios deseadas
id_rest_final = id_rest[~id_rest["category"].isin(no)]
id_rest_final.head(5)

,gmap_id,category
2,0x80c2c778e3b73d33:0xbdc58662a4a97d49,Korean restaurant
6,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,Restaurant
8,0x87ec235c54d25b31:0x3b75fb5facc602f,Health food restaurant
68,0x89c6c89efcaed69d:0xded973f6033e7dba,Restaurant
75,0x87fd0e70c5f5d87b:0xdf340eeb75040ef3,Bar


In [13]:
# Se observa que las categorías de negocios se encuentren dentro de lo esperado (bares y restaurantes).
id_rest_final["category"].unique()

array(['Korean restaurant', 'Restaurant', 'Health food restaurant', 'Bar',
       'Buffet restaurant', 'Cocktail bar', 'Fast food restaurant',
       'Family restaurant', 'Seafood restaurant', 'Dominican restaurant',
       'Taco restaurant', 'Mexican restaurant', 'Restaurant or cafe',
       'Bar & grill', 'Breakfast restaurant', 'Lunch restaurant',
       'Asian restaurant', 'Hot pot restaurant', 'Chinese restaurant',
       'Mediterranean restaurant', 'Takeout Restaurant', 'Sports bar',
       'Italian restaurant', 'Wine bar', 'Ramen restaurant',
       'American restaurant', 'Modern French restaurant',
       'Pizza restaurant', 'Vegetarian restaurant', 'Delivery Restaurant',
       'Sushi restaurant', 'Japanese restaurant', 'Creole restaurant',
       'Caribbean restaurant', 'Barbecue restaurant', 'Snack bar',
       'Hamburger restaurant', 'Soul food restaurant',
       'Burrito restaurant', 'Gay bar', 'Latin American restaurant',
       'Spanish restaurant', 'Cheesesteak restaur

In [15]:
# Se filtra el data de los negocios para obtener aquellos que son de negocios de restaurantes o bares.
dfg_rest = dfg_sites[dfg_sites["gmap_id"].isin(id_rest_final["gmap_id"])]
dfg_rest.head(10)

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
2,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,None,34.058092,-118.292130,[Korean restaurant],4.4,18,None,"[[Thursday, 6:30AM–6PM], [Friday, 6:30AM–6PM],...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 6PM,"[0x80c2c78249aba68f:0x35bf16ce61be751d, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
6,Vons Chicken,"Vons Chicken, 12740 La Mirada Blvd, La Mirada,...",0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,None,33.916402,-118.010855,[Restaurant],4.5,18,None,"[[Thursday, 11AM–9:30PM], [Friday, 11AM–9:30PM...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 9:30PM,None,https://www.google.com/maps/place//data=!4m2!3...
8,"Sweet Rewards Gluten Free Bakery, LLC","Sweet Rewards Gluten Free Bakery, LLC, 85 NE D...",0x87ec235c54d25b31:0x3b75fb5facc602f,None,41.616079,-93.865487,"[Bakery, Health food restaurant]",4.7,21,None,"[[Thursday, 10AM–5:30PM], [Friday, 10AM–5:30PM...","{'Accessibility': None, 'Activities': None, 'A...",Permanently closed,"[0x87ee974869295555:0x95f310d065882c9b, 0x87ec...",https://www.google.com/maps/place//data=!4m2!3...
68,Vivi Bubble Tea,"Vivi Bubble Tea, 701 S 5th St, Philadelphia, P...",0x89c6c89efcaed69d:0xded973f6033e7dba,"Bubble tea shop offering hot & cold versions, ...",39.940293,-75.150923,[Restaurant],4.0,8,None,None,"{'Accessibility': None, 'Activities': None, 'A...",None,"[0x89c6c62a40f4b733:0x73bfeec1898f272b, 0x89c6...",https://www.google.com/maps/place//data=!4m2!3...
75,The Bar Stool,"The Bar Stool, 618 Main St, Highland, WI 53543",0x87fd0e70c5f5d87b:0xdf340eeb75040ef3,None,43.046428,-90.380139,[Bar],4.5,18,None,None,"{'Accessibility': None, 'Activities': None, 'A...",None,"[0x87fd0e70dc5eeb3d:0x7f3ce1daba0a5f24, 0x87fd...",https://www.google.com/maps/place//data=!4m2!3...
96,Hale Pops,"Hale Pops, 55-370 Kamehameha Hwy, Laie, HI 96762",0x7c00456eecad3111:0x8217f9600c51f33,None,21.637796,-157.920714,[Restaurant],4.4,18,None,"[[Thursday, 11AM–8PM], [Friday, 11AM–8PM], [Sa...",{'Accessibility': ['Wheelchair accessible entr...,Closed ⋅ Opens 11AM,"[0x7c00451360f80cf1:0x930291a38bab3132, 0x7c00...",https://www.google.com/maps/place//data=!4m2!3...
114,Cape Seafood Shack,"Cape Seafood Shack, 603 Del Prado Blvd S, Cape...",0x88db4147b1d9e6f3:0x943dbd10a92ba1b1,None,26.641377,-81.940545,[Restaurant],5.0,1,$$,None,{'Accessibility': ['Wheelchair accessible entr...,None,None,https://www.google.com/maps/place//data=!4m2!3...
115,Fresh Point Country Buffet,"Fresh Point Country Buffet, 10525 US-19, Pinel...",0x88c2e4e34f1ed783:0x76c5da381c499d79,None,27.867489,-82.702971,[Buffet restaurant],5.0,2,None,"[[Thursday, 11AM–9PM], [Friday, 11AM–9PM], [Sa...",{'Accessibility': ['Wheelchair accessible entr...,Permanently closed,"[0x88c2e5593fbd022b:0x2ccd7a719849c9f7, 0x88c2...",https://www.google.com/maps/place//data=!4m2!3...
123,Golden Castle,"Golden Castle, 1906 E 12th St, Austin, TX 78702",0x8644b59b8fe872e5:0x5e638876caa84cc3,None,30.273985,-97.719563,[Restaurant],4.5,8,None,"[[Thursday, 5PM–12AM], [Friday, 5PM–12AM], [Sa...",{'Accessibility': ['Wheelchair accessible entr...,Closed ⋅ Opens 5PM,None,https://www.google.com/maps/place//data=!4m2!3...
126,Onyx Bar,"Onyx Bar, 11011 W Charleston Blvd, Las Vegas, ...",0x80c8be4e73e8263f:0x3edb275a351e6266,None,36.156205,-115.332636,[Cocktail bar],4.7,8,None,"[[Thursday, 4–11PM], [Friday, 4PM–1AM], [Satur...",{'Accessibility': ['Wheelchair accessible entr...,Closed ⋅ Opens 4PM,"[0x80c8be531e3b3471:0xb681abb0b8d07596, 0x80c8...",https://www.google.com/maps/place//data=!4m2!3...


In [16]:
# Se guarda la base de restaurantes y bares.
dfg_rest.to_parquet("../Data/Google/reviews-estados-parquet/dfg_rest.parquet")

## Conclusiones 
========================================================

* Se realiza la lectura, concatenación y exportación de los datos de locales de google
* Debido a la cuantiosa cantidad de información, el preprocesamiento de estos datos se realiza en el EDA

# ETL de Google - 2

## Carga de datos 
==================================================================

In [17]:
# Sites
dfg_rest = pd.read_parquet("../Data/Google/reviews-estados-parquet/dfg_rest.parquet")

# Reviews
# Este dataset se carga más adelante en su sección de pre-procesamiento

## Preprocesamiento
============================================================

### Dataset SITES

In [18]:
print(dfg_rest.info())
dfg_rest.sample(2)

<class 'pandas.core.frame.DataFrame'>
Index: 248852 entries, 2 to 3025006
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   name              248852 non-null  object 
 1   address           247751 non-null  object 
 2   gmap_id           248852 non-null  object 
 3   description       79018 non-null   object 
 4   latitude          248852 non-null  float64
 5   longitude         248852 non-null  float64
 6   category          248852 non-null  object 
 7   avg_rating        248852 non-null  float64
 8   num_of_reviews    248852 non-null  int64  
 9   price             115831 non-null  object 
 10  hours             220683 non-null  object 
 11  MISC              246706 non-null  object 
 12  state             221749 non-null  object 
 13  relative_results  203743 non-null  object 
 14  url               248852 non-null  object 
dtypes: float64(3), int64(1), object(11)
memory usage: 30.4+ MB
None


,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
1496641,CALVIN'S KOREAN HOT CHICKEN,"CALVIN'S KOREAN HOT CHICKEN, 3211 Holiday Ct S...",0x80dc07e35b78cb4f:0x96beb1a46c73711e,"Compact, informal destination with outdoor sea...",32.870019,-117.233508,"[Fried chicken takeaway, Chicken shop, Chicken...",5.0,46,None,"[[Tuesday, 11AM–8PM], [Wednesday, 11AM–8PM], [...",{'Accessibility': ['Wheelchair accessible entr...,Closed ⋅ Opens 11AM Wed,None,https://www.google.com/maps/place//data=!4m2!3...
2668084,Berwick Family Resturaunt,"Berwick Family Resturaunt, 110 N Mulberry St, ...",0x89c59d0325812891:0x6fc3f2d46857fbaf,None,41.054444,-76.235556,[Family restaurant],4.3,138,None,"[[Sunday, 6AM–2PM], [Monday, 8AM–2PM], [Tuesda...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 2PM,"[0x89c59d1a4ccc1899:0xe605a9205d997bbb, 0x89c5...",https://www.google.com/maps/place//data=!4m2!3...


In [19]:
# Adecuación de tipos de dato
dfg_rest = dfg_rest[['name', 'address', 'gmap_id', 'latitude', 'longitude', 'avg_rating', 'num_of_reviews', 'price']]
dfg_rest = dfg_rest.rename(columns={'address': 'address_full'})
dfg_rest[['address', 'city', 'postal_code']] = dfg_rest['address_full'].str.extract(r'.*,\s*([^,]+),\s*([^,]+),\s*([^,]+)')
dfg_rest['state'] = dfg_rest['postal_code'].str.split(' ').str[0]
dfg_rest['postal_code'] = dfg_rest['postal_code'].str.split(' ').str[1]

dfg_rest['postal_code'] = pd.to_numeric(dfg_rest['postal_code'], errors='coerce')
dfg_rest = dfg_rest.dropna(subset=['postal_code'])
dfg_rest['postal_code'] = dfg_rest['postal_code'].astype(int)

In [20]:
print(dfg_rest.info())
dfg_rest.sample(2)

<class 'pandas.core.frame.DataFrame'>
Index: 231427 entries, 2 to 3025006
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   name            231427 non-null  object 
 1   address_full    231427 non-null  object 
 2   gmap_id         231427 non-null  object 
 3   latitude        231427 non-null  float64
 4   longitude       231427 non-null  float64
 5   avg_rating      231427 non-null  float64
 6   num_of_reviews  231427 non-null  int64  
 7   price           104084 non-null  object 
 8   address         231427 non-null  object 
 9   city            231427 non-null  object 
 10  postal_code     231427 non-null  int32  
 11  state           231427 non-null  object 
dtypes: float64(3), int32(1), int64(1), object(7)
memory usage: 22.1+ MB
None


,name,address_full,gmap_id,latitude,longitude,avg_rating,num_of_reviews,price,address,city,postal_code,state
2101489,Difrego's Hide Out Charcoal Grill Kitchen & Pizza,Difrego's Hide Out Charcoal Grill Kitchen & Pi...,0x89c3bf603982c725:0x70b8bc0dd32dd5b4,40.554275,-74.524840,3.7,58,None,77 S Main St,South Bound Brook,8880,NJ
1109016,Zab Thai Food Kitchen,"Zab Thai Food Kitchen, 16720 E Ave of the Foun...",0x872b99f02649ba2f:0x1f7cf9580074a2c9,33.604968,-111.719693,4.4,25,None,16720 E Ave of the Fountains,Fountain Hills,85268,AZ


In [21]:
# Convertir precio a formato numerico
price_mapping = {'$': 1, '$$': 2, '$$$': 3, '$$$$': 4}
dfg_rest['price'] = dfg_rest['price'].map(price_mapping)

# Reemplazar valores nulos del campo "precio" con el promedio de precios basado en cercanía, utilzando el código postal
dfg_rest['postal_code'] = dfg_rest['postal_code'].astype(str)
average_price_by_zip = dfg_rest.groupby('postal_code')['price'].apply(lambda x: x.dropna().astype(float).mean())
dfg_rest['price'] = dfg_rest.apply(lambda row: average_price_by_zip.get(row['postal_code']) if pd.isnull(row['price']) else row['price'], axis=1)

dfg_rest = dfg_rest.dropna(subset=['price'])

In [22]:
print(dfg_rest.info())
dfg_rest.sample(2)

<class 'pandas.core.frame.DataFrame'>
Index: 223485 entries, 2 to 3025006
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   name            223485 non-null  object 
 1   address_full    223485 non-null  object 
 2   gmap_id         223485 non-null  object 
 3   latitude        223485 non-null  float64
 4   longitude       223485 non-null  float64
 5   avg_rating      223485 non-null  float64
 6   num_of_reviews  223485 non-null  int64  
 7   price           223485 non-null  float64
 8   address         223485 non-null  object 
 9   city            223485 non-null  object 
 10  postal_code     223485 non-null  object 
 11  state           223485 non-null  object 
dtypes: float64(4), int64(1), object(7)
memory usage: 22.2+ MB
None


,name,address_full,gmap_id,latitude,longitude,avg_rating,num_of_reviews,price,address,city,postal_code,state
2954365,Pizza Hut,"Pizza Hut, 2601 Castle Hayne Rd, Wilmington, N...",0x89a98a995db410b7:0x27606ef77269faa3,34.289313,-77.923051,3.9,148,1.000000,2601 Castle Hayne Rd,Wilmington,28401,NC
40757,Banh Mi - Juice and Sandwich,"Banh Mi - Juice and Sandwich, 2731 FM 1463 #80...",0x86412380b6f37c41:0x469da66a0a137bc1,29.750890,-95.850806,4.3,28,1.382353,2731 FM 1463 #800,Katy,77494,TX


In [26]:
# Subida del dataset a GCS
destino = 'dfg_rest.parquet'
dataframe_to_parquet_and_upload(dfg_rest,bucket_name,destino)

#dfg_rest.to_parquet('dfg_rest.parquet')

Archivo temp.parquet subido a dfg_rest.parquet.


#### Creacion del dataset de categorias

In [29]:
dfg_rest.head()

,name,address_full,gmap_id,latitude,longitude,avg_rating,num_of_reviews,price,address,city,postal_code,state
2,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.292130,4.4,18,1.411765,761 S Vermont Ave,Los Angeles,90005,CA
6,Vons Chicken,"Vons Chicken, 12740 La Mirada Blvd, La Mirada,...",0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,33.916402,-118.010855,4.5,18,1.333333,12740 La Mirada Blvd,La Mirada,90638,CA
8,"Sweet Rewards Gluten Free Bakery, LLC","Sweet Rewards Gluten Free Bakery, LLC, 85 NE D...",0x87ec235c54d25b31:0x3b75fb5facc602f,41.616079,-93.865487,4.7,21,1.500000,85 NE Dartmoor Dr,Waukee,50263,IA
68,Vivi Bubble Tea,"Vivi Bubble Tea, 701 S 5th St, Philadelphia, P...",0x89c6c89efcaed69d:0xded973f6033e7dba,39.940293,-75.150923,4.0,8,1.581818,701 S 5th St,Philadelphia,19147,PA
75,The Bar Stool,"The Bar Stool, 618 Main St, Highland, WI 53543",0x87fd0e70c5f5d87b:0xdf340eeb75040ef3,43.046428,-90.380139,4.5,18,1.000000,618 Main St,Highland,53543,WI


In [30]:
dfg_restaurants = pd.read_parquet("../Data/Google/reviews-estados-parquet/dfg_rest.parquet")
dfg_categories = dfg_restaurants[['gmap_id', 'category']]
dfg_categories = dfg_categories.explode('category')
dfg_categories.rename(columns={'gmap_id': 'site_id'}, inplace=True)

dfg_categories.info()
dfg_categories.sample(10)

<class 'pandas.core.frame.DataFrame'>
Index: 626995 entries, 2 to 3025006
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   site_id   626995 non-null  object
 1   category  626995 non-null  object
dtypes: object(2)
memory usage: 14.4+ MB


,site_id,category
441362,0x89c2fdad033831ab:0x7b53ace9a1db0b8e,Cocktail bar
636921,0x86456c86118a668f:0xdc3ac1088a2d1d71,Bar
1609424,0x89c3b10b567f818d:0x6eec95cec36523ce,Restaurant
767957,0x866f94219f0ecf8d:0xac7168f5b68d3b26,Bar
2301257,0x808fc7556dac75b7:0xcecc805f4ac5f192,Pizza delivery
2937141,0x88f5af4b9e73cbfd:0x60ffec905c4a188f,Cookie shop
585482,0x865c667da63be485:0x65a6f731c4a8d363,Bar
2102017,0x89c808f3141a34ed:0x23d8d02f51f3a878,Breakfast restaurant
2416913,0x88351bc9dc742ab1:0xc898b37481f7d862,Mexican restaurant
1779069,0x8091e232b91fc16d:0xb0fa83c9a8c4eddd,Frozen yogurt shop


In [31]:
dfg_categories_grouped = dfg_categories.groupby(['site_id'])['category'].count().reset_index()
dfg_categories_grouped.sort_values(by='category', ascending=False, inplace=True)
dfg_categories_grouped.head()

,site_id,category
24829,0x808f7792bd0c683d:0xefefd9d96f8fcb11,22
100420,0x87c5696c2304eb11:0xf60dbc11df643d20,22
211649,0x89c28801576a92a1:0xe8183528656d5f08,22
224911,0x89c6cf1e78429c4d:0xffccfa7176afdc7e,20
224636,0x89c6c9d5df403af7:0xa304c9f7a08ac8e3,20


In [32]:
# Subida del dataset a GCS
destino = 'dfg_categories.parquet'
dataframe_to_parquet_and_upload(dfg_categories,bucket_name,destino)

# Exportar
#dfg_categories.to_parquet('dfg_site_categories.parquet')
#dfg_categories.to_excel('dfg_site_categories.xlsx')

Archivo temp.parquet subido a dfg_categories.parquet.


#### Creacion del dataset de atributos

In [24]:
dfg_restaurants = pd.read_parquet("../Data/Google/reviews-estados-parquet/dfg_rest.parquet")
dfg_misc = dfg_restaurants[['gmap_id', 'MISC']]

dfg_misc.rename(columns={'gmap_id': 'site_id'}, inplace=True)

dfg_misc.info()
dfg_misc.sample(10)

<class 'pandas.core.frame.DataFrame'>
Index: 248852 entries, 2 to 3025006
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   site_id  248852 non-null  object
 1   MISC     246706 non-null  object
dtypes: object(2)
memory usage: 5.7+ MB


,site_id,MISC
216315,0x80c2c77c69813cf1:0x2c6387e3b9c87f35,"{'Accessibility': None, 'Activities': None, 'A..."
2409639,0x808580d4c7a21d4f:0xb2ee66b91c20d28c,{'Accessibility': ['Wheelchair-accessible entr...
2631981,0x890068f93f4d2143:0xd3d3f1cdae4d568,{'Accessibility': ['Wheelchair-accessible entr...
530520,0x54906ab4a6788555:0x46c9fbcad1334459,{'Accessibility': ['Wheelchair accessible entr...
1608852,0x89b7ba614d71abcd:0x6ab8d0874354633f,"{'Accessibility': None, 'Activities': None, 'A..."
823499,0x5340d34e777e9f83:0xf36b7d18ba02cb3b,{'Accessibility': ['Wheelchair accessible entr...
1203834,0x8640bfd907e9298f:0x681a58578a4ffb3e,"{'Accessibility': None, 'Activities': None, 'A..."
1076288,0x89a521aa698c30d1:0x58e81f8e9c1e112a,"{'Accessibility': None, 'Activities': None, 'A..."
1029266,0x89c25baee1bd2ad7:0xaa66666c49748685,"{'Accessibility': None, 'Activities': None, 'A..."
562468,0x87cf4531158bf035:0xbcf8231c7c968fc8,{'Accessibility': ['Wheelchair accessible entr...


In [25]:
# Se obtiene un dataframe con los datos normalizados de "MISC"
x = pd.json_normalize(dfg_misc["MISC"])

#Se concatenan los dos dataframes.
dfg_misc = pd.concat([dfg_misc.reset_index(drop=True), x.reset_index(drop=True)], axis=1)

# Se elimina la columna "MISC"
dfg_misc.drop(columns=["MISC"],inplace=True)

# Se visualiza el resultado.
dfg_misc.head(5)

,site_id,Accessibility,Activities,Amenities,Atmosphere,Crowd,Dining options,From the business,Getting here,Health & safety,Health and safety,Highlights,Lodging options,Offerings,Payments,Planning,Popular for,Recycling,Service options
0,0x80c2c778e3b73d33:0xbdc58662a4a97d49,[Wheelchair accessible entrance],None,[Good for kids],[Casual],None,None,None,NaN,None,None,None,NaN,[Comfort food],None,None,None,None,"[Takeout, Dine-in, Delivery]"
1,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,[Wheelchair accessible entrance],None,[Good for kids],[Casual],[College students],"[Lunch, Dinner]",None,NaN,"[Mask required, Staff wear masks, Staff requir...",None,None,NaN,"[Comfort food, Quick bite, Small plates]",None,None,"[Lunch, Dinner, Solo dining]",None,"[Outdoor seating, Curbside pickup, No-contact ..."
2,0x87ec235c54d25b31:0x3b75fb5facc602f,None,None,None,None,None,None,None,NaN,None,None,None,NaN,None,None,None,None,None,[Delivery]
3,0x89c6c89efcaed69d:0xded973f6033e7dba,None,None,[Good for kids],[Casual],"[College students, Locals]",None,None,NaN,None,None,None,NaN,None,None,None,None,None,[Delivery]
4,0x87fd0e70c5f5d87b:0xdf340eeb75040ef3,None,None,[Restroom],"[Casual, Cozy]",[Groups],None,None,NaN,None,None,"[Fast service, Great beer selection, Sports]",NaN,"[Alcohol, Beer, Hard liquor]",None,None,None,None,"[Dine-in, Delivery]"


In [26]:
# Se procede a pasar los datos de las columnas accesibility, activities y amenities a diferentes dataframes 
dfg_attributes_accessibility = dfg_misc[["site_id","Accessibility"]]
dfg_attributes_activities = dfg_misc[["site_id","Activities"]]
dfg_attributes_amenities = dfg_misc[["site_id","Amenities"]]

# Se eliminan datos nulos
dfg_attributes_accessibility.dropna(inplace=True)
dfg_attributes_activities.dropna(inplace=True)
dfg_attributes_amenities.dropna(inplace=True)

# Se explotan los datos dentro de las columnas de las categorías
dfg_attributes_accessibility = dfg_attributes_accessibility.explode("Accessibility")
dfg_attributes_activities = dfg_attributes_activities.explode("Activities")
dfg_attributes_amenities = dfg_attributes_amenities.explode("Amenities")

# Se resetean índices para asegurarse de que sean únicos
dfg_attributes_accessibility = dfg_attributes_accessibility.reset_index(drop=True)
dfg_attributes_activities = dfg_attributes_activities.reset_index(drop=True)
dfg_attributes_amenities = dfg_attributes_amenities.reset_index(drop=True)

In [27]:
# Se renombran las columnas
dfg_attributes_accessibility = dfg_attributes_accessibility.rename(columns={"Accessibility":"attributes"})
dfg_attributes_activities = dfg_attributes_activities.rename(columns={"Activities":"attributes"})
dfg_attributes_amenities = dfg_attributes_amenities.rename(columns={"Amenities":"attributes"})

In [28]:
# Se unen los datos en el mismo dataframe
dfg_attributes = pd.concat([dfg_attributes_accessibility, 
                         dfg_attributes_activities, 
                         dfg_attributes_amenities], axis=0, ignore_index=True)
dfg_attributes.head()

,site_id,attributes
0,0x80c2c778e3b73d33:0xbdc58662a4a97d49,Wheelchair accessible entrance
1,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,Wheelchair accessible entrance
2,0x7c00456eecad3111:0x8217f9600c51f33,Wheelchair accessible entrance
3,0x88db4147b1d9e6f3:0x943dbd10a92ba1b1,Wheelchair accessible entrance
4,0x88db4147b1d9e6f3:0x943dbd10a92ba1b1,Wheelchair accessible parking lot


In [29]:
# Subida del dataset a GCS
destino = 'dfg_attributes.parquet'
dataframe_to_parquet_and_upload(dfg_attributes,bucket_name,destino)

# dfg_attributes.head()
# dfg_attributes.to_csv("dfg_attributes.csv")

Archivo temp.parquet subido a dfg_attributes.parquet.


#### Creacion del dataset de precios

In [30]:
dfg_rest_prices_by_zip = dfg_rest.groupby('postal_code')['price'].apply(lambda x: x.dropna().astype(float).mean())

dfg_rest_prices_by_zip.reset_index()

,postal_code,price
0,1,1.000000
1,10001,1.639344
2,10002,1.710744
3,10003,1.828025
4,10004,1.750000
...,...,...
17612,99829,2.000000
17613,99833,2.000000
17614,99835,2.000000
17615,99901,1.444444


In [31]:
dfg_rest_prices_by_zip_df = dfg_rest_prices_by_zip.to_frame()

In [32]:
# Subida del dataset a GCS
destino = 'dfg_rest_prices_by_zip.parquet'
dataframe_to_parquet_and_upload(dfg_rest_prices_by_zip_df,bucket_name,destino)

# dfg_rest_prices_by_zip_df.to_parquet('dfg_rest_prices_by_zip.parquet')

Archivo temp.parquet subido a dfg_rest_prices_by_zip.parquet.


#### Creación de dataset de coordenadas

In [20]:
dfg_rest = pd.read_parquet("gs://archivos-preprocesados-henry/dfg_rest.parquet")
dfg_rest.sample()

,name,address_full,gmap_id,latitude,longitude,avg_rating,num_of_reviews,price,address,city,postal_code,state
458409,Oriental Express Cafe,"Oriental Express Cafe, 158 S San Gabriel Blvd,...",0x80c2dade68c7cbc7:0x7ff6dee975a83f26,34.101703,-118.090615,3.9,78,1.0,158 S San Gabriel Blvd,San Gabriel,91776,CA


In [21]:
dfg_rest_coord = dfg_rest[['gmap_id', 'latitude', 'longitude', 'name', 'state', 'city', 'postal_code',]]
dfg_rest_coord = dfg_rest_coord.rename(columns={'gmap_id': 'business_id'})
dfg_rest_coord['source'] = 'google'

print(dfg_rest_coord.info())
dfg_rest_coord.sample(2)

<class 'pandas.core.frame.DataFrame'>
Index: 223485 entries, 2 to 3025006
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   business_id  223485 non-null  object 
 1   latitude     223485 non-null  float64
 2   longitude    223485 non-null  float64
 3   name         223485 non-null  object 
 4   state        223485 non-null  object 
 5   city         223485 non-null  object 
 6   postal_code  223485 non-null  object 
 7   source       223485 non-null  object 
dtypes: float64(2), object(6)
memory usage: 15.3+ MB
None


,business_id,latitude,longitude,name,state,city,postal_code,source
598857,0x54956cf11ccc41bd:0x3a1ab2ea2d9433c5,45.356030,-122.841043,503 Uncorked,OR,Sherwood,97140,google
210070,0x89b7b683ecd5ff59:0xfbf765df032dbb4d,38.886239,-77.097635,Park Lane Tavern of Clarendon,VA,Arlington,22201,google


In [22]:
# Función para convertir latitud y longitud a coordenadas cartesianas
def lat_lon_to_cartesian(lat, lon):
    R = 6371  # Radio de la Tierra en kilómetros
    x = R * math.cos(math.radians(lat)) * math.cos(math.radians(lon))
    y = R * math.cos(math.radians(lat)) * math.sin(math.radians(lon))
    return x, y

# Suponiendo que tienes un DataFrame llamado df con las columnas business_id, latitud y longitud
# Agrega campos de coordenadas cartesianas x e y al DataFrame
dfg_rest_coord['x'], dfg_rest_coord['y'] = zip(*dfg_rest_coord.apply(lambda row: lat_lon_to_cartesian(row['latitude'], row['longitude']), axis=1))

print(dfg_rest_coord.info())
dfg_rest_coord.sample(2)

<class 'pandas.core.frame.DataFrame'>
Index: 223485 entries, 2 to 3025006
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   business_id  223485 non-null  object 
 1   latitude     223485 non-null  float64
 2   longitude    223485 non-null  float64
 3   name         223485 non-null  object 
 4   state        223485 non-null  object 
 5   city         223485 non-null  object 
 6   postal_code  223485 non-null  object 
 7   source       223485 non-null  object 
 8   x            223485 non-null  float64
 9   y            223485 non-null  float64
dtypes: float64(4), object(6)
memory usage: 18.8+ MB
None


,business_id,latitude,longitude,name,state,city,postal_code,source,x,y
1818645,0x87c53902a263273d:0xc8475f1049f7577b,37.652076,-92.667064,Godfather's Pizza,MO,Lebanon,65536,google,-234.715247,-5038.678262
508680,0x8800212a53d9637f:0x70c57ba6bdd44e63,44.960121,-89.628021,Townies Grill,WI,Wausau,54403,google,29.267613,-4508.016790


In [23]:
# Subida del dataset a GCS
destino = 'dfg_rest_coord.parquet'
dataframe_to_parquet_and_upload(dfg_rest_coord,bucket_name,destino)

# Exportar
#dfg_rest_coord.to_parquet('dfg_rest_coord.parquet', index=False)
#dfg_rest_coord.to_excel('dfg_rest_coord.xlsx', index=False)

Archivo temp.parquet subido a dfg_rest_coord.parquet.


### Dataset REVIEW

In [5]:
dfg_rest = pd.read_parquet("../Data/Google/reviews-estados-parquet/dfg_rest.parquet")

dfg_rest_ids = dfg_rest[['gmap_id', 'state']]

print(dfg_rest_ids.info())
dfg_rest_ids.sample(2)

<class 'pandas.core.frame.DataFrame'>
Index: 248852 entries, 2 to 3025006
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   gmap_id  248852 non-null  object
 1   state    221749 non-null  object
dtypes: object(2)
memory usage: 5.7+ MB
None


,gmap_id,state
2690910,0x87d8cd57e918db7f:0x7c4d0340d0685691,None
1791413,0x88f8a943a7176b1f:0x9ac19a8fd5f3ec33,Permanently closed


In [6]:
# Lista para almacenar los dataframes filtrados
dataframes_filtrados_rest = []

# Función para procesar cada archivo JSON
def procesar_archivo_json(filepath):
    try:
        df = pd.read_json(filepath, lines=True)
        # Filtrar las filas según los gmap_ids deseados
        df_filtrado = df[df['gmap_id'].isin(dfg_rest_ids["gmap_id"])]
        dataframes_filtrados_rest.append(df_filtrado)
    except Exception as e:
        print(f"Error al procesar el archivo {filepath}: {e}")

In [156]:
# Carga de datos desde los datasets .json para cada estado de USA

#states = [
#    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 
#    'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 
#    'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 
#    'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 
#    'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY',
#    'District_of_Columbia'
#]
#df_list = []

#for state in states:
#    filename = f'/Users/Juan/Documents/Academics/DataScience/PjFinal/Google/GoogleReviews/revgoo_{state}.parquet'
#    dfgrev = pd.read_parquet(filename)
#    dfgrev = dfgrev.dropna(subset=['text'])

    # Filtrado por restaurantes
#    dfgrev = dfgrev[dfgrev['gmap_id'].isin(dfg_rest_ids['gmap_id'])]
    # Análisis de sentimientos con VADER
#    analyzer = SentimentIntensityAnalyzer()
#    dfgrev['vader_polarity'] = dfgrev['text'].apply(lambda text: analyzer.polarity_scores(text)['compound'])
#    dfgrev['vader_sentiment'] = pd.cut(dfgrev['vader_polarity'], bins=[-float('inf'), -0.001, 0.0, float('inf')], labels=[-1, 0, 1])
    # Seleccion de campos
#    dfgrev = dfgrev[['gmap_id', 'state', 'user_id', 'time', 'rating', 'vader_polarity', 'vader_sentiment']]

#    df_list.append(dfgrev)
#dfg_reviews = pd.concat(df_list, ignore_index=True)

# print(dfg_reviews.shape)
# dfg_reviews.sample(2)

(7162236, 8)


,user_id,name,time,rating,text,gmap_id,vader_polarity,vader_sentiment
336158,1.089925e+20,Keaton Weinbrenner,1606236458720,4,good service and good pizza,0x87d28d5aa640a9c5:0x3c9cc5d3a99de80c,0.7003,1
2822320,1.035633e+20,Jon Swedberg,1531232260236,1,None,0x89e47d7f337876d7:0xbc34ba9e5ea67023,0.0000,0


In [7]:
# Diccionario de estados y rutas
estados_ruta = {
    "AL": "../Data/Google/reviews-estados/review-Alabama/review-Alabama",
    'AK':'../Data/Google/reviews-estados/review-Alaska/review-Alaska',
    'AZ':'../Data/Google/reviews-estados/review-Arizona/review-Arizona',
    'AR':'../Data/Google/reviews-estados/review-Arkansas/review-Arkansas',
    'CA':'../Data/Google/reviews-estados/review-California/review-California',
    'CO':'../Data/Google/reviews-estados/review-Colorado/review-Colorado',
    'CT':'../Data/Google/reviews-estados/review-Connecticut/review-Connecticut',
    'DE':"../Data/Google/reviews-estados/review-Delaware/review-Delaware",
    'District_of_Columbia':"../Data/Google/reviews-estados/review-District_of_Columbia",
    'FL':"../Data/Google/reviews-estados/review-Florida/review-Florida",
    'GA':"../Data/Google/reviews-estados/review-Georgia/review-Georgia",
    'HI':"../Data/Google/reviews-estados/review-Hawaii/review-Hawaii",
    'ID':"../Data/Google/reviews-estados/review-Idaho/review-Idaho",
    'IL':"../Data/Google/reviews-estados/review-Illinois/review-Illinois",
    'IN':"../Data/Google/reviews-estados/review-Indiana/review-Indiana",
    'IA':"../Data/Google/reviews-estados/review-Iowa/review-Iowa",
    'KS':"../Data/Google/reviews-estados/review-Kansas/review-Kansas",
    'KY':"../Data/Google/reviews-estados/review-Kentucky/review-Kentucky",
    'LA':"../Data/Google/reviews-estados/review-Louisiana/review-Louisiana",
    'ME':"../Data/Google/reviews-estados/review-Maine/review-Maine",
    'MD':"../Data/Google/reviews-estados/review-Maryland/review-Maryland",
    'MA':"../Data/Google/reviews-estados/review-Massachusetts/review-Massachusetts",
    'MI':"../Data/Google/reviews-estados/review-Michigan/review-Michigan",
    'MN':"../Data/Google/reviews-estados/review-Minnesota/review-Minnesota",
    'MS':"../Data/Google/reviews-estados/review-Mississippi/review-Mississippi",
    'MO':"../Data/Google/reviews-estados/review-Missouri/review-Missouri",
    'MT':"../Data/Google/reviews-estados/review-Montana/review-Montana",
    'NE':"../Data/Google/reviews-estados/review-Nebraska/review-Nebraska",
    'NV':"../Data/Google/reviews-estados/review-Nevada/review-Nevada",
    'NH':"../Data/Google/reviews-estados/review-New_Hampshire/review-New_Hampshire",
    'NJ':"../Data/Google/reviews-estados/review-New_Jersey/review-New_Jersey",
    'NM':"../Data/Google/reviews-estados/review-New_Mexico/review-New_Mexico",
    'NY':"../Data/Google/reviews-estados/review-New_York/review-New_York",
    'NC':"../Data/Google/reviews-estados/review-North_Carolina/review-North_Carolina",
    'ND':"../Data/Google/reviews-estados/review-North_Dakota/review-North_Dakota",
    'OH':"../Data/Google/reviews-estados/review-Ohio/review-Ohio",
    'OK':"../Data/Google/reviews-estados/review-Oklahoma/review-Oklahoma",
    'OR':"../Data/Google/reviews-estados/review-Oregon/review-Oregon",
    'PA':"../Data/Google/reviews-estados/review-Pennsylvania/review-Pennsylvania",
    'RI':"../Data/Google/reviews-estados/review-Rhode_Island/review-Rhode_Island",
    'SC':"../Data/Google/reviews-estados/review-South_Carolina/review-South_Carolina",
    'SD':"../Data/Google/reviews-estados/review-South_Dakota/review-South_Dakota",
    'TN':"../Data/Google/reviews-estados/review-Tennessee/review-Tennessee",
    'TX':"../Data/Google/reviews-estados/review-Texas/review-Texas",
    'UT':"../Data/Google/reviews-estados/review-Utah/review-Utah",
    'VT':"../Data/Google/reviews-estados/review-Vermont/review-Vermont",
    'VA':"../Data/Google/reviews-estados/review-Virginia/review-Virginia",
    'WA':"../Data/Google/reviews-estados/review-Washington/review-Washington",
    'WV':"../Data/Google/reviews-estados/review-West_Virginia/review-West_Virginia",
    'WI':"../Data/Google/reviews-estados/review-Wisconsin/review-Wisconsin",
    'WY':"../Data/Google/reviews-estados/review-Wyoming/review-Wyoming"
}

# Iterar sobre el diccionario y procesar los archivos
for state, ruta_directorio in estados_ruta.items():
    # Procesar cada archivo en el directorio
    for filename in os.listdir(ruta_directorio):
        if filename.endswith('.json'):
            filepath = os.path.join(ruta_directorio, filename)
            procesar_archivo_json(filepath)


# Concatenar todos los dataframes en uno solo
dfg_reviews = pd.concat(dataframes_filtrados_rest, ignore_index=True)

In [8]:
# Se eliminan los campos de pics y resp.
dfg_reviews.drop(columns=["pics","resp"],inplace=True)

In [9]:
dfg_reviews.head()

,user_id,name,time,rating,text,gmap_id
0,1.179759e+20,Anthony Roberts,1463443013514,4,"On the higher end of price for pizza, but they...",0x8889221157fb3455:0x5c125c40c3eccc2a
1,1.143165e+20,Jonathan Robert,1447623939865,3,"Food was ok, felt like the atmosphere as well ...",0x8889221157fb3455:0x5c125c40c3eccc2a
2,1.136326e+20,Brian Harvey,1469293549247,4,"Good food, service so so",0x8889221157fb3455:0x5c125c40c3eccc2a
3,1.076840e+20,Ashley Maddox,1442279219480,5,Love it,0x8889221157fb3455:0x5c125c40c3eccc2a
4,1.126036e+20,West Martin,1382634896130,5,Yum,0x8889221157fb3455:0x5c125c40c3eccc2a


In [10]:
# preprocesado reviews 
dfg_reviews.to_parquet("../Data/Google/reviews-estados-parquet/dfg_reviews_preprocesados.parquet")

In [33]:
dfg_reviews = pd.read_parquet("../Data/Google/reviews-estados-parquet/dfg_reviews_preprocesados.parquet")
dfg_reviews.head()

,user_id,name,time,rating,text,gmap_id
0,1.179759e+20,Anthony Roberts,1463443013514,4,"On the higher end of price for pizza, but they...",0x8889221157fb3455:0x5c125c40c3eccc2a
1,1.143165e+20,Jonathan Robert,1447623939865,3,"Food was ok, felt like the atmosphere as well ...",0x8889221157fb3455:0x5c125c40c3eccc2a
2,1.136326e+20,Brian Harvey,1469293549247,4,"Good food, service so so",0x8889221157fb3455:0x5c125c40c3eccc2a
3,1.076840e+20,Ashley Maddox,1442279219480,5,Love it,0x8889221157fb3455:0x5c125c40c3eccc2a
4,1.126036e+20,West Martin,1382634896130,5,Yum,0x8889221157fb3455:0x5c125c40c3eccc2a


In [34]:
dfg_reviews_usa = dfg_reviews

In [35]:
dfg_reviews_usa['time'] = dfg_reviews_usa['time'] / 1000

# Convertir la marca de tiempo a un objeto de fecha y hora
dfg_reviews_usa['datetime'] = dfg_reviews_usa['time'].apply(lambda x: datetime.fromtimestamp(x))

dfg_reviews_usa.head()

,user_id,name,time,rating,text,gmap_id,datetime
0,1.179759e+20,Anthony Roberts,1.463443e+09,4,"On the higher end of price for pizza, but they...",0x8889221157fb3455:0x5c125c40c3eccc2a,2016-05-16 18:56:53.514
1,1.143165e+20,Jonathan Robert,1.447624e+09,3,"Food was ok, felt like the atmosphere as well ...",0x8889221157fb3455:0x5c125c40c3eccc2a,2015-11-15 16:45:39.865
2,1.136326e+20,Brian Harvey,1.469294e+09,4,"Good food, service so so",0x8889221157fb3455:0x5c125c40c3eccc2a,2016-07-23 12:05:49.247
3,1.076840e+20,Ashley Maddox,1.442279e+09,5,Love it,0x8889221157fb3455:0x5c125c40c3eccc2a,2015-09-14 20:06:59.480
4,1.126036e+20,West Martin,1.382635e+09,5,Yum,0x8889221157fb3455:0x5c125c40c3eccc2a,2013-10-24 12:14:56.130


In [36]:
dfg_reviews_sent = dfg_reviews_usa

# Análisis de sentimientos con VADER
analyzer = SentimentIntensityAnalyzer()

# Aplicar análisis de sentimientos, manejando valores nulos
dfg_reviews_sent['vader_polarity'] = dfg_reviews_sent['text'].apply(
    lambda text: analyzer.polarity_scores(text)['compound'] if pd.notnull(text) else 0
)

# Clasificar sentimientos según la polaridad
dfg_reviews_sent['vader_sentiment'] = pd.cut(
    dfg_reviews_sent['vader_polarity'], 
    bins=[-float('inf'), -0.001, 0.0, float('inf')], 
    labels=[-1, 0, 1]
)  

dfg_reviews_sent.head()

,user_id,name,time,rating,text,gmap_id,datetime,vader_polarity,vader_sentiment
0,1.179759e+20,Anthony Roberts,1.463443e+09,4,"On the higher end of price for pizza, but they...",0x8889221157fb3455:0x5c125c40c3eccc2a,2016-05-16 18:56:53.514,0.3898,1
1,1.143165e+20,Jonathan Robert,1.447624e+09,3,"Food was ok, felt like the atmosphere as well ...",0x8889221157fb3455:0x5c125c40c3eccc2a,2015-11-15 16:45:39.865,0.6275,1
2,1.136326e+20,Brian Harvey,1.469294e+09,4,"Good food, service so so",0x8889221157fb3455:0x5c125c40c3eccc2a,2016-07-23 12:05:49.247,0.4404,1
3,1.076840e+20,Ashley Maddox,1.442279e+09,5,Love it,0x8889221157fb3455:0x5c125c40c3eccc2a,2015-09-14 20:06:59.480,0.6369,1
4,1.126036e+20,West Martin,1.382635e+09,5,Yum,0x8889221157fb3455:0x5c125c40c3eccc2a,2013-10-24 12:14:56.130,0.0000,0


In [37]:
# Se obtiene el dataset final con las columnas requeridas.
dfg_reviews = dfg_reviews_usa[['user_id', 'gmap_id', 'datetime', 'rating', 'vader_polarity', 'vader_sentiment']]
dfg_reviews.head()

,user_id,gmap_id,datetime,rating,vader_polarity,vader_sentiment
0,1.179759e+20,0x8889221157fb3455:0x5c125c40c3eccc2a,2016-05-16 18:56:53.514,4,0.3898,1
1,1.143165e+20,0x8889221157fb3455:0x5c125c40c3eccc2a,2015-11-15 16:45:39.865,3,0.6275,1
2,1.136326e+20,0x8889221157fb3455:0x5c125c40c3eccc2a,2016-07-23 12:05:49.247,4,0.4404,1
3,1.076840e+20,0x8889221157fb3455:0x5c125c40c3eccc2a,2015-09-14 20:06:59.480,5,0.6369,1
4,1.126036e+20,0x8889221157fb3455:0x5c125c40c3eccc2a,2013-10-24 12:14:56.130,5,0.0000,0


In [38]:
print(dfg_reviews.info())
dfg_reviews.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12948989 entries, 0 to 12948988
Data columns (total 6 columns):
 #   Column           Dtype         
---  ------           -----         
 0   user_id          float64       
 1   gmap_id          object        
 2   datetime         datetime64[ns]
 3   rating           int64         
 4   vader_polarity   float64       
 5   vader_sentiment  category      
dtypes: category(1), datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 506.3+ MB
None


,user_id,gmap_id,datetime,rating,vader_polarity,vader_sentiment
3226041,1.122316e+20,0x88fb7f8646dc04e1:0x94cf57df55c08968,2019-12-21 13:36:29.281,5,0.9467,1
3844739,1.099641e+20,0x880f2a8b9e00a5f9:0x4090c1304f4b52b0,2019-01-22 23:45:55.123,4,0.9124,1
614728,1.007731e+20,0x862da4a85e6bdd57:0x3dfccdbf88535d41,2021-02-03 22:21:22.387,5,0.0000,0
3869713,1.171513e+20,0x880e31ddbe12cbff:0x5ddffb9b1d258497,2020-02-17 06:53:55.275,4,0.0000,0
10979363,1.161979e+20,0x886467a7dc19cc5f:0x4fb51f2d35798228,2019-05-02 21:47:26.905,5,0.8883,1


In [39]:
# Subida del dataset a GCS
destino = 'dfg_reviews.parquet'
dataframe_to_parquet_and_upload(dfg_reviews,bucket_name,destino)

# Exportación de datos
#dfg_reviews_usa.to_parquet('dfgrevall.parquet')

Archivo temp.parquet subido a dfg_reviews.parquet.


## Conclusiones

* Dataset SITES
    * Conversión del campo "precio" a formato numérico
    * Obtención de los campos "state" y "postal_code" a partir del campo "address" - esto será fundamental para combinar luego el dataset de google y yelp
    * Creación del dataset de categorías a partir del campo "category" cuyos valores originales están en formato tipo "lista" - será de utilidad para el modelo de machine learning
    * Creación del dataset de atributos a partir del campo "MISC" cuyos valores originales están en formato tipo "diccionario de listas" - será de utilidad para el modelo de machine learning
    * Creación del dataset de precios - será utilizado para crear el campo "precio" en el dataset de yelp
    * Creación del dataset de coordenadas - se utilizará para concatenar un listado único de sites entre ambos datasets de google y yelp
* Dataset REVIEWS
    * Mediante un código se leen todos los archivos de reviews de cada estado, concatenando los registros que correspondan a restaurantes

# ETL Yelp - 1

## Carga de datos
=======================================================================================

In [5]:
# Yelp
business = pd.read_pickle("../Data/Yelp/business.pkl")
checkin = pd.read_json('../Data/Yelp/checkin.json', lines=True)
user = pd.read_parquet('../Data/Yelp/user.parquet')

In [6]:
# Base de datos externa
df_uszip = pd.read_excel('../Data/dt_uszips.xlsx')

## Preprocesamiento
====================================================================================

### Dataset uszip (externo)

In [7]:
print(df_uszip.info())
df_uszip.sample(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33787 entries, 0 to 33786
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   postal_code       33787 non-null  int64  
 1   city              33787 non-null  object 
 2   state             33787 non-null  object 
 3   state_name        33787 non-null  object 
 4   population        33770 non-null  float64
 5   density           33770 non-null  float64
 6   county_fips       33787 non-null  int64  
 7   county_name       33787 non-null  object 
 8   county_weights    33787 non-null  object 
 9   county_names_all  33787 non-null  object 
 10  county_fips_all   33787 non-null  object 
 11  timezone          33787 non-null  object 
dtypes: float64(2), int64(2), object(8)
memory usage: 3.1+ MB
None


,postal_code,city,state,state_name,population,density,county_fips,county_name,county_weights,county_names_all,county_fips_all,timezone
13916,42350,Island,KY,Kentucky,1269.0,11.8,21149,McLean,"{""21149"": 100}",McLean,21149,America/Chicago
10073,30573,Tallulah Falls,GA,Georgia,114.0,22.1,13241,Rabun,"{""13241"": 100}",Rabun,13241,America/New_York


In [8]:
df_uszip = df_uszip[['postal_code', 'state']]
df_uszip['postal_code'] = df_uszip['postal_code'].astype(str)

df_uszip.info()
df_uszip.sample(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33787 entries, 0 to 33786
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   postal_code  33787 non-null  object
 1   state        33787 non-null  object
dtypes: object(2)
memory usage: 528.1+ KB


,postal_code,state
4220,14549,NY
1025,4017,ME


In [9]:
# Se sube a la nube.
destino = 'df_uszip.parquet'
dataframe_to_parquet_and_upload(df_uszip,bucket_name,destino)

Archivo temp.parquet subido a df_uszip.parquet.


### Dataset BUSINESS

In [9]:
print(business.info())
business.sample(5)

<class 'pandas.core.frame.DataFrame'>
Index: 150346 entries, 0 to 150345
Data columns (total 28 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   business_id   150346 non-null  object
 1   name          150346 non-null  object
 2   address       150346 non-null  object
 3   city          150346 non-null  object
 4   state         150343 non-null  object
 5   postal_code   150346 non-null  object
 6   latitude      150346 non-null  object
 7   longitude     150346 non-null  object
 8   stars         150346 non-null  object
 9   review_count  150346 non-null  object
 10  is_open       150346 non-null  object
 11  attributes    136602 non-null  object
 12  categories    150243 non-null  object
 13  hours         127123 non-null  object
 14  business_id   5 non-null       object
 15  name          5 non-null       object
 16  address       5 non-null       object
 17  city          5 non-null       object
 18  state         5 non-null     

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
19106,UeNG6Lj1K0_F5BAQvmswBQ,La Vang Solar Nail,"3251 Manhattan Blvd, Ste 3",Harvey,FL,70058,29.861569,-90.046449,2.5,19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106836,rjozeYvawecmdKbwOghh5w,Water Tower Dental Group and Orthodontics,2231 Michigan Ave,Arnold,PA,63010,38.439038,-90.377512,4.0,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60487,mAum-FMG31OF-oJq8Xq6tA,Enterprise Car Sales,3313 E Speedway Blvd,Tucson,DE,85716,32.236635,-110.921957,4.5,26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99449,3MvWBlTqt41Zbw5_DQ2YkQ,Meridian Smiles Dentistry and Orthodontics,"2274 N Eagle Rd, Ste 130",Meridian,FL,83646,43.625363,-116.353907,4.5,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123716,HLZfvmlAt4K91MCQXvvwCQ,Nails Rap 3,"5095 N La Canada Dr, Ste 101",Tucson,PA,85704,32.298981,-110.996675,3.0,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
business_copy = business.copy()
business_copy = business_copy.loc[:,~business_copy.columns.duplicated()]
business_copy.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,NaN,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,NaN,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [11]:
# Selección de campos
dfybsn = business_copy[['business_id', 'name', 'state', 'city', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count', 'is_open']]

# Eliminación de columnas duplicadas
dfybsn = dfybsn.loc[:,~dfybsn.columns.duplicated()]

# Adcuación de tipos de datos
dfybsn['latitude'] = pd.to_numeric(dfybsn['latitude'], errors='coerce')
dfybsn['longitude'] = pd.to_numeric(dfybsn['longitude'], errors='coerce')
dfybsn['stars'] = pd.to_numeric(dfybsn['stars'], errors='coerce')
dfybsn['review_count'] = pd.to_numeric(dfybsn['review_count'], errors='coerce')
dfybsn['is_open'] = pd.to_numeric(dfybsn['is_open'], errors='coerce')

# Eliminación de duplicados
dfybsn.drop_duplicates()

'''Reemplazo de valores del campo "state": los valores del dataset original no se corresponden con los códigos postales, 
por lo cual se utilizará una base de datos externa para corregir esta serie.'''

dfybsn.drop(columns=['state'], inplace=True)
dfybsn = pd.merge(dfybsn, df_uszip, on='postal_code', how='left')

# Campos auxiliares
dfybsn.dropna(subset=['state'], inplace=True)
dfybsn['state_city'] = dfybsn['state'].str.cat(dfybsn['city'], sep=' - ')
dfybsn['city_postalcode'] = dfybsn['city'].str.cat(dfybsn['postal_code'], sep=' - ')
dfybsn['state_city_postalcode'] = dfybsn['state'].str.cat(dfybsn['city'], sep=' - ').str.cat(dfybsn['postal_code'], sep=' - ')

dfybsn.info()
dfybsn.sample(2)

<class 'pandas.core.frame.DataFrame'>
Index: 135158 entries, 0 to 150345
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   business_id            135158 non-null  object 
 1   name                   135158 non-null  object 
 2   city                   135158 non-null  object 
 3   postal_code            135158 non-null  object 
 4   latitude               135158 non-null  float64
 5   longitude              135158 non-null  float64
 6   stars                  135158 non-null  float64
 7   review_count           135158 non-null  int64  
 8   is_open                135158 non-null  int64  
 9   state                  135158 non-null  object 
 10  state_city             135158 non-null  object 
 11  city_postalcode        135158 non-null  object 
 12  state_city_postalcode  135158 non-null  object 
dtypes: float64(3), int64(2), object(8)
memory usage: 14.4+ MB


,business_id,name,city,postal_code,latitude,longitude,stars,review_count,is_open,state,state_city,city_postalcode,state_city_postalcode
59727,SUARAtRB_3ZcLLcH8f_F9Q,Ybor Chophouse,Tampa,33605,27.960428,-82.441648,3.5,479,0,FL,FL - Tampa,Tampa - 33605,FL - Tampa - 33605
110798,abmP4I_2mlXu4uM3Gmfgbw,El Perron de Obregon,Tucson,85705,32.294759,-111.012549,4.5,8,1,AZ,AZ - Tucson,Tucson - 85705,AZ - Tucson - 85705


In [24]:
# Subida del dataset a GCS
destino = 'dfybsn.parquet'
dataframe_to_parquet_and_upload(dfybsn,bucket_name,destino)

Archivo temp.parquet subido a dfybsn.parquet.


### Creación de dataset RESTAURANTES

In [12]:
# Selección de campos
dfybct = business_copy[['business_id', 'categories']]

# Eliminación de columnas duplicadas
dfybct = dfybct.loc[:,~dfybct.columns.duplicated()]

# Adecuación de campos
dfybct = dfybct.explode('categories')
dfybct = dfybct.assign(categories=dfybct['categories'].str.split(', ')).explode('categories')
dfybct.reset_index(drop=True, inplace=True)

# Eliminación de nulos
dfybct = dfybct.dropna(subset=['categories'])

dfybct.info()
dfybct.sample(2)

<class 'pandas.core.frame.DataFrame'>
Index: 668592 entries, 0 to 668694
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   business_id  668592 non-null  object
 1   categories   668592 non-null  object
dtypes: object(2)
memory usage: 15.3+ MB


,business_id,categories
616378,YRBCEpAGjedZVC_6Dw8S0A,Bars
383381,yy5-YZpI5cdV9NcnOrLLoQ,Food


In [13]:
# Se obtienen los id de los negocios cuyas categorías incluyan restaurantes o bares.
id_rest = dfybct[dfybct["categories"].str.contains('restaurant|bar|food', case=False)]
id_rest.head(10)

,business_id,categories
17,MTSW4McQd7CbVtyjqoe9mw,Restaurants
18,MTSW4McQd7CbVtyjqoe9mw,Food
24,mWMc6_wTdE0EUBKIGXDVfA,Food
26,CF33F8-E6oudUQ46HnavjQ,Fast Food
28,CF33F8-E6oudUQ46HnavjQ,Food
30,CF33F8-E6oudUQ46HnavjQ,Restaurants
40,k0hlBqXX-Bt0vf1op7Jr1w,Restaurants
42,k0hlBqXX-Bt0vf1op7Jr1w,Bars
47,bBDDEgkFA1Otx9Lfe7BZUQ,Fast Food
49,bBDDEgkFA1Otx9Lfe7BZUQ,Restaurants


In [14]:
# Se observan las categorías obtenidas
id_rest["categories"].unique()

array(['Restaurants', 'Food', 'Fast Food', 'Bars', 'Food Trucks',
       'Sushi Bars', 'Wine Bars', 'Seafood', 'Cocktail Bars',
       'Specialty Food', 'Juice Bars & Smoothies', 'Sports Bars',
       'Beer Bar', 'Food Delivery Services', 'Barbers',
       'Do-It-Yourself Food', 'Live/Raw Food', 'Barbeque', 'Hookah Bars',
       'Soul Food', 'Ethnic Food', 'Food Tours', 'Comfort Food',
       'Food Stands', 'Seafood Markets', 'Whiskey Bars', 'Imported Food',
       'Barre Classes', 'Bar Crawl', 'Dive Bars', 'Tapas Bars',
       'Gay Bars', 'Food Court', 'Bartenders', 'Tiki Bars', 'Cabaret',
       'Piano Bars', 'Pop-Up Restaurants', 'Cigar Bars',
       'Bartending Schools', 'Restaurant Supplies', 'Food Banks',
       'Champagne Bars', 'Drive-Thru Bars',
       'Undersea/Hyperbaric Medicine', 'Vermouth Bars', 'Oxygen Bars',
       'Hotel bar', 'Beach Bars'], dtype=object)

In [15]:
# Se seleccionan las categorías que no pertenecen a los atributos esperados.
no = ['Barbers',"Food Tours",'Barre Classes','Bartenders','Cabaret'
      ,'Bartending Schools','Restaurant Supplies','Food Banks','Undersea/Hyperbaric Medicine']

In [16]:
# Se filtra nuevamente el dataset de las categorías para obtener únicamente las categorías de los negocios deseadas
id_rest_final = id_rest[~id_rest["categories"].isin(no)]
id_rest_final.head(5)

,business_id,categories
17,MTSW4McQd7CbVtyjqoe9mw,Restaurants
18,MTSW4McQd7CbVtyjqoe9mw,Food
24,mWMc6_wTdE0EUBKIGXDVfA,Food
26,CF33F8-E6oudUQ46HnavjQ,Fast Food
28,CF33F8-E6oudUQ46HnavjQ,Food


In [17]:
# Se obtienen los negocios con categorías en Restaurantes y bares
dfyrst = dfybsn[dfybsn["business_id"].isin(id_rest_final["business_id"])]

In [18]:
# Se obtienen las categorías completas de los negocios identificados
dfybct = dfybct[dfybct["business_id"].isin(dfyrst["business_id"])]

In [19]:
dfybct.info()
dfybct.sample(10)

<class 'pandas.core.frame.DataFrame'>
Index: 273644 entries, 17 to 668667
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   business_id  273644 non-null  object
 1   categories   273644 non-null  object
dtypes: object(2)
memory usage: 6.3+ MB


,business_id,categories
393136,NbyLOoqWrXGE0NKh8DdmUA,Gelato
540443,ack-0MAGQnipu0y9SN9Vng,American (Traditional)
163631,rvbqvQod-USx8EpjXtJNdw,American (New)
398565,HhcLmjO4OltuaqS2mOe8FA,Chinese
266337,gaoXL657-gkwVXynq3LjeQ,Vietnamese
550020,_HMrtN8oEoC4arLmVVSZmw,Restaurants
662572,MCSyG3ndsNFJN-F9MQWM0A,Sandwiches
44520,w520vYWEpEYBf90rSr9NVA,Seafood
550920,GUH9h8yQzSY3s8P3Nssidg,Fast Food
549249,S0UPup0pLE9knjrwc-kAdw,Food


In [20]:
dfy_site_categories = dfybct.copy()
dfy_site_categories.rename(columns={"business_id":"site_id"}, inplace=True)

# Subida del dataset de categoría a GCS
destino = 'dfy_rest_categories.parquet'
dataframe_to_parquet_and_upload(dfy_site_categories,bucket_name,destino)

# dfy_site_categories.to_parquet("dfy_site_categories.parquet")

Archivo temp.parquet subido a dfy_rest_categories.parquet.


In [21]:
# Eliminación de duplicados
dfyrst = dfyrst.drop_duplicates(subset='business_id', keep='first')

dfyrst.info()
dfyrst.sample(2)

<class 'pandas.core.frame.DataFrame'>
Index: 59148 entries, 3 to 150339
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   business_id            59148 non-null  object 
 1   name                   59148 non-null  object 
 2   city                   59148 non-null  object 
 3   postal_code            59148 non-null  object 
 4   latitude               59148 non-null  float64
 5   longitude              59148 non-null  float64
 6   stars                  59148 non-null  float64
 7   review_count           59148 non-null  int64  
 8   is_open                59148 non-null  int64  
 9   state                  59148 non-null  object 
 10  state_city             59148 non-null  object 
 11  city_postalcode        59148 non-null  object 
 12  state_city_postalcode  59148 non-null  object 
dtypes: float64(3), int64(2), object(8)
memory usage: 6.3+ MB


,business_id,name,city,postal_code,latitude,longitude,stars,review_count,is_open,state,state_city,city_postalcode,state_city_postalcode
7108,ZhCnKe7U1TsHWnAlXl8Isw,Old Standard Fried Chicken,Saint Louis,63110,38.620026,-90.256022,3.0,195,0,MO,MO - Saint Louis,Saint Louis - 63110,MO - Saint Louis - 63110
25638,sScHtSXYOvlAdaup1kQfNw,Burger King,Tampa,33619,27.928870,-82.376087,1.5,10,1,FL,FL - Tampa,Tampa - 33619,FL - Tampa - 33619


In [22]:
# Agregado de campo "precio", basado en el dataset de google
dfg_rest_prices_by_zip_data = "gs://archivos-preprocesados-henry/dfg_rest_prices_by_zip.parquet"
dfg_rest_prices_by_zip = pd.read_parquet(dfg_rest_prices_by_zip_data)

dfg_rest_prices_by_zip.sample(5)

dfyrst = pd.merge(dfyrst, dfg_rest_prices_by_zip, on=['postal_code'], how='left')

dfyrst.info()
dfyrst.sample(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1991330 entries, 0 to 1991329
Data columns (total 24 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   business_id            object 
 1   name_x                 object 
 2   city_x                 object 
 3   postal_code            object 
 4   latitude_x             float64
 5   longitude_x            float64
 6   stars                  float64
 7   review_count           int64  
 8   is_open                int64  
 9   state_x                object 
 10  state_city             object 
 11  city_postalcode        object 
 12  state_city_postalcode  object 
 13  name_y                 object 
 14  address_full           object 
 15  gmap_id                object 
 16  latitude_y             float64
 17  longitude_y            float64
 18  avg_rating             float64
 19  num_of_reviews         float64
 20  price                  float64
 21  address                object 
 22  city_y            

,business_id,name_x,city_x,postal_code,latitude_x,longitude_x,stars,review_count,is_open,state_x,...,address_full,gmap_id,latitude_y,longitude_y,avg_rating,num_of_reviews,price,address,city_y,state_y
846249,Qhigd6zj7sXQ7oFN551Tmg,Ding Tea Fowler,Tampa,33612,28.055128,-82.441685,4.5,18,0,FL,...,"McAlister's Deli, 11402 N 30th St, Tampa, FL 3...",0x88c2c7a92bc1be63:0xc49847c691dfd27,28.052256,-82.426670,4.0,6.0,2.000000,11402 N 30th St,Tampa,FL
101744,ovLgaPIMrx7PDZPrEpTMvg,Mei's World Pizza,Tampa,33613,28.080764,-82.431900,4.5,117,1,FL,...,"D'Alessio Pizza & Pastries, 14831 N Florida Av...",0x88c2c75b830ec3fb:0x7ef881d17cd34808,28.083660,-82.459118,5.0,3.0,1.142857,14831 N Florida Ave,Tampa,FL


In [23]:
# Subida del dataset a GCS
destino = 'dfy_rest.parquet'
dataframe_to_parquet_and_upload(dfyrst,bucket_name,destino)

# dfyrst.to_parquet('dfy_rest.parquet')

Archivo temp.parquet subido a dfy_rest.parquet.


### Creación de dataset de coordenadas

In [25]:
dfyrst_coord = dfyrst[['business_id', 'latitude_x', 'longitude_x', 'name_x', 'state_x', 'city_x', 'postal_code']]

dfyrst_coord['source'] = 'yelp'
dfyrst_coord.sample(2)

print(dfyrst_coord.info())
dfyrst_coord.sample(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1991330 entries, 0 to 1991329
Data columns (total 8 columns):
 #   Column       Dtype  
---  ------       -----  
 0   business_id  object 
 1   latitude_x   float64
 2   longitude_x  float64
 3   name_x       object 
 4   state_x      object 
 5   city_x       object 
 6   postal_code  object 
 7   source       object 
dtypes: float64(2), object(6)
memory usage: 121.5+ MB
None


,business_id,latitude_x,longitude_x,name_x,state_x,city_x,postal_code,source
506084,jx1Qc-qPxIvTYP2ewWkRWg,32.223627,-110.965240,Dillinger Brewing Company,AZ,Tucson,85705,yelp
962620,yL9j5ASNic_DaDKf9lwMhw,29.929647,-90.077213,Stein's Market and Deli,LA,New Orleans,70130,yelp


In [27]:
# Función para convertir latitud y longitud a coordenadas cartesianas
def lat_lon_to_cartesian(lat, lon):
    R = 6371  # Radio de la Tierra en kilómetros
    x = R * math.cos(math.radians(lat)) * math.cos(math.radians(lon))
    y = R * math.cos(math.radians(lat)) * math.sin(math.radians(lon))
    return x, y

# Suponiendo que tienes un DataFrame llamado df con las columnas business_id, latitud y longitud
# Agrega campos de coordenadas cartesianas x e y al DataFrame
dfyrst_coord['x'], dfyrst_coord['y'] = zip(*dfyrst_coord.apply(lambda row: lat_lon_to_cartesian(row['latitude_x'], row['longitude_x']), axis=1))

print(dfyrst_coord.info())
dfyrst_coord.sample(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1991330 entries, 0 to 1991329
Data columns (total 10 columns):
 #   Column       Dtype  
---  ------       -----  
 0   business_id  object 
 1   latitude_x   float64
 2   longitude_x  float64
 3   name_x       object 
 4   state_x      object 
 5   city_x       object 
 6   postal_code  object 
 7   source       object 
 8   x            float64
 9   y            float64
dtypes: float64(4), object(6)
memory usage: 151.9+ MB
None


,business_id,latitude_x,longitude_x,name_x,state_x,city_x,postal_code,source,x,y
1421139,pN4bu3X4Upie2fc9H7BY7Q,39.986984,-75.405698,MOD Pizza,PA,Newtown Square,19073,yelp,1229.981456,-4723.897252
824472,xzfKqIPAf0jqyUWJG9cNNA,40.073265,-75.305205,Ted's Place Pizzeria & Restaurant,PA,Conshohocken,19428,yelp,1236.699539,-4715.763740


In [28]:
# Subida del dataset a GCS
destino = 'dfyrst_coord.parquet'
dataframe_to_parquet_and_upload(dfyrst_coord,bucket_name,destino)


# Exportar 
#dfyrst_coord.to_parquet('dfyrst_coord.parquet', index=False)
#dfyrst_coord.to_excel('dfyrst_coord.xlsx', index=False)

Archivo temp.parquet subido a dfyrst_coord.parquet.


### Creación de dataset BUSINESS ATTRIBUTES

In [29]:
# Selección de campos
dfybat = business[['business_id','attributes']]

# Eliminación de columnas duplicadas
dfybat = dfybat.loc[:,~dfybat.columns.duplicated()]

dfybat.info()
dfybat.sample(5)

<class 'pandas.core.frame.DataFrame'>
Index: 150346 entries, 0 to 150345
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   business_id  150346 non-null  object
 1   attributes   136602 non-null  object
dtypes: object(2)
memory usage: 3.4+ MB


,business_id,attributes
19498,TdOidl1ipY_HZvCeZwBEuA,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo..."
136833,S1v_v8rIfq7nCuOHfWuwFQ,"{'BusinessAcceptsCreditCards': 'True', 'WiFi':..."
103196,OZi0fOn8-ISUwhEAWC_8Gg,"{'WheelchairAccessible': 'True', 'Alcohol': 'u..."
3420,Shx_E96rNccpZNBl6l2Eyw,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc..."
135890,X-nOvoQNrB2bHZp7AyXlvw,"{'Alcohol': 'u'beer_and_wine'', 'HasTV': 'True..."


In [30]:
# Adecuación de campos
dfybat = dfybat.explode('attributes')

# Eliminación de duplicados
dfybat.drop_duplicates()

# Eliminación de nulos
dfybat = dfybat.dropna(subset=['attributes'])

In [31]:
dfybat.info()
dfybat.sample(5)

<class 'pandas.core.frame.DataFrame'>
Index: 1206820 entries, 0 to 150345
Data columns (total 2 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   business_id  1206820 non-null  object
 1   attributes   1206820 non-null  object
dtypes: object(2)
memory usage: 27.6+ MB


,business_id,attributes
119232,vqq18oM8_gsfl24ndr2LLQ,OutdoorSeating
67903,e4InIycH2PAJWccBBj0tAA,RestaurantsTakeOut
31420,AmS4jlZqK09tCVfzULubHA,RestaurantsTakeOut
144539,ziGS7TkxebMgoqcYVMcloQ,BusinessAcceptsCreditCards
69349,JRPblBSca5VQH8ai6yTjRA,WheelchairAccessible


In [32]:
# Subida del dataset a GCS
destino = 'dfy_attributes.parquet'
dataframe_to_parquet_and_upload(dfybat,bucket_name,destino)

#dfybat[['business_id', 'attributes']].to_parquet('dfy_attributes.parquet')
#dfybat[['business_id', 'attributes']].to_csv('dfy_attributes.csv')

Archivo temp.parquet subido a dfy_attributes.parquet.


### Precios

In [ ]:
'''# Filtrar los valores del campo "attributes" relacionados con la palabra "precio"
df_prices = dfybat[dfybat['attributes'].str.contains('price', case=False)]
df_prices.rename(columns={'attributes': 'price'}, inplace=True)

print(df_prices['price'].nunique())
# Mostrar el DataFrame filtrado
df_prices.sample(2)'''
# Se ha encontrado que este campo no contiene información de precios ni de rangos de precios

### Dataset CHECKIN

In [33]:
print(checkin.shape)
checkin.head(2)

(131930, 2)


,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
1,--0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011..."


In [34]:
# Filtrado por restarurants y agregado de características
dfychk = pd.merge(checkin, dfyrst, on='business_id', how='inner')

dfychk.info()
dfychk.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1948397 entries, 0 to 1948396
Data columns (total 25 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   business_id            object 
 1   date                   object 
 2   name_x                 object 
 3   city_x                 object 
 4   postal_code            object 
 5   latitude_x             float64
 6   longitude_x            float64
 7   stars                  float64
 8   review_count           int64  
 9   is_open                int64  
 10  state_x                object 
 11  state_city             object 
 12  city_postalcode        object 
 13  state_city_postalcode  object 
 14  name_y                 object 
 15  address_full           object 
 16  gmap_id                object 
 17  latitude_y             float64
 18  longitude_y            float64
 19  avg_rating             float64
 20  num_of_reviews         float64
 21  price                  float64
 22  address           

,business_id,date,name_x,city_x,postal_code,latitude_x,longitude_x,stars,review_count,is_open,...,address_full,gmap_id,latitude_y,longitude_y,avg_rating,num_of_reviews,price,address,city_y,state_y
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020...",Frankie's Raw Bar,New Port Richey,34652,28.217288,-82.733344,4.5,24,1,...,"Netta's Hideaway & Café, 6229 Grand Blvd, Port...",0x88c291cef93494c9:0xf2664ecbbd1e7a40,28.249400,-82.719733,2.8,8.0,1.454545,6229 Grand Blvd,Port Richey,FL
1,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020...",Frankie's Raw Bar,New Port Richey,34652,28.217288,-82.733344,4.5,24,1,...,"Circle K, 5523 FL-54, New Port Richey, FL 34652",0x88c291e6bf2d5987:0xd24458e36441f2db,28.217706,-82.722683,2.8,8.0,1.454545,5523 FL-54,New Port Richey,FL


In [36]:
dfychk['date'] = pd.to_datetime(dfychk['date'], errors='coerce')
dfychk['year'] = dfychk['date'].dt.year
dfychk['month'] = dfychk['date'].dt.month

# Filtrado cronológico (los datos están entre dic2009 y ene2022, por lo tanto se quitan los años 2009 y 2022)
dfychk = dfychk[(dfychk['year'] >= 2010) & (dfychk['year'] <= 2021)]

dfychk.info()
dfychk.sample(2)

C:\Users\dmon2\AppData\Local\Temp\ipykernel_2868\806472024.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dfychk['date'] = pd.to_datetime(dfychk['date'], errors='coerce')


<class 'pandas.core.frame.DataFrame'>
Index: 35857 entries, 422 to 1948396
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   business_id            35857 non-null  object        
 1   date                   35857 non-null  datetime64[ns]
 2   name_x                 35857 non-null  object        
 3   city_x                 35857 non-null  object        
 4   postal_code            35857 non-null  object        
 5   latitude_x             35857 non-null  float64       
 6   longitude_x            35857 non-null  float64       
 7   stars                  35857 non-null  float64       
 8   review_count           35857 non-null  int64         
 9   is_open                35857 non-null  int64         
 10  state_x                35857 non-null  object        
 11  state_city             35857 non-null  object        
 12  city_postalcode        35857 non-null  object        
 13  st

,business_id,date,name_x,city_x,postal_code,latitude_x,longitude_x,stars,review_count,is_open,...,latitude_y,longitude_y,avg_rating,num_of_reviews,price,address,city_y,state_y,year,month
250713,70Kj7pob7w21cw3z0lvJnA,2011-06-20 23:42:11,Southeast Body Shop,Tucson,85719,32.209351,-110.958103,3.0,5,1,...,32.232523,-110.951557,3.5,8.0,1.416667,1303 E University Blvd,Tucson,AZ,2011.0,6.0
1535058,mShRkXrlpWJMkTz0dWj5VA,2019-08-22 22:30:48,Vituperio Artisan Breads & Studio,Lansdale,19446,40.208114,-75.297553,5.0,5,1,...,40.233524,-75.272064,4.1,148.0,2.000000,612 E Main St,Lansdale,PA,2019.0,8.0


In [37]:
# Subida del dataset a GCS
destino = 'dfychk.parquet'
dataframe_to_parquet_and_upload(dfychk,bucket_name,destino)

#dfychk.to_parquet('dfy_checkins.parquet')

Archivo temp.parquet subido a dfychk.parquet.


### Dataset USER

In [38]:
user.info()
user.sample(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2105597 entries, 0 to 2105596
Data columns (total 22 columns):
 #   Column              Dtype  
---  ------              -----  
 0   user_id             object 
 1   name                object 
 2   review_count        int64  
 3   yelping_since       object 
 4   useful              int64  
 5   funny               int64  
 6   cool                int64  
 7   elite               object 
 8   friends             object 
 9   fans                int64  
 10  average_stars       float64
 11  compliment_hot      int64  
 12  compliment_more     int64  
 13  compliment_profile  int64  
 14  compliment_cute     int64  
 15  compliment_list     int64  
 16  compliment_note     int64  
 17  compliment_plain    int64  
 18  compliment_cool     int64  
 19  compliment_funny    int64  
 20  compliment_writer   int64  
 21  compliment_photos   int64  
dtypes: float64(1), int64(16), object(5)
memory usage: 353.4+ MB


,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
680169,T2iAME3eMX5LmMGoHDWhUw,Beth,134,2013-08-21 23:50:06,105,14,15,,"SkTfptyQGBQV5EuH4i_Ebg, 49lKFP9Z5uG53-hjs0_7mg...",1,...,0,0,0,0,2,0,1,1,0,0
1430768,ZdRguxCCofZqd9wWgfgW9g,Domenic,14,2013-01-12 02:52:58,135,40,92,,"zayZ0KmjDgxRfiWrr1EA1g, Ya3nkNGGR6O3rXnVtUwUJQ...",32,...,0,0,0,0,0,0,0,0,0,0


In [39]:
# Selección de campos
dfyusr = user[['user_id', 'review_count', 'fans', 'yelping_since', 'useful', 'funny', 'cool', 'average_stars']]

dfyusr['yelping_since'] = pd.to_datetime(dfyusr['yelping_since'], errors='coerce')
dfyusr.reset_index(drop=True, inplace=True)

dfyusr['year'] = dfyusr['yelping_since'].dt.year
dfyusr['month'] = dfyusr['yelping_since'].dt.month
dfyusr['year_month'] = dfyusr['year'].astype(str).str.slice(-2) + dfyusr['month'].astype(str).str.zfill(2)

dfyusr.info()
dfyusr.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2105597 entries, 0 to 2105596
Data columns (total 11 columns):
 #   Column         Dtype         
---  ------         -----         
 0   user_id        object        
 1   review_count   int64         
 2   fans           int64         
 3   yelping_since  datetime64[ns]
 4   useful         int64         
 5   funny          int64         
 6   cool           int64         
 7   average_stars  float64       
 8   year           int32         
 9   month          int32         
 10  year_month     object        
dtypes: datetime64[ns](1), float64(1), int32(2), int64(5), object(2)
memory usage: 160.6+ MB


,user_id,review_count,fans,yelping_since,useful,funny,cool,average_stars,year,month,year_month
388680,LfBw_x6u5xGiIIFX5mTFrg,7,0,2018-07-17 23:46:31,2,0,0,3.57,2018,7,1807
1811034,0L_tQhb8HuigIpUI-viD1Q,2,0,2013-06-09 16:55:53,1,0,0,5.00,2013,6,1306
1550504,x9Pbvg8UFzmh0MKqo_zsFA,2,0,2011-06-14 01:28:24,0,0,0,5.00,2011,6,1106
1475697,i7YVlk9TPlDe4wxAzV3lcw,7,0,2014-08-12 03:37:56,18,0,4,2.60,2014,8,1408
525123,tl2Uhj2wBGS5AuqZDnyhCw,12,0,2015-06-21 03:26:31,4,0,1,4.00,2015,6,1506


In [40]:
# Subida del dataset a GCS
destino = 'dfyusr.parquet'
dataframe_to_parquet_and_upload(dfyusr,bucket_name,destino)


# dfyusr.to_parquet("dfy_user.parquet")

Archivo temp.parquet subido a dfyusr.parquet.


## Conclusiones

* Dataset BUSINESS
   * Se corrigen los códigos postales originales pues estaban erróneos
   * Se eliminan columnas duplicadas
   * Se adecúan los tipos de datos para su posterior procesamiento
   * Se eliminan registros duplicados
* Creación de dataset RESTAURANTES
    Mediante un archivo que contiene las categorías deseadas de restaurantes y bares, se filtran los locales a considerar
    Se agrega el campo precio tomado del dataset de google, con el cual se estima el rango de precios del local tomando el promedio del código postal al que pertenece
* Creación del dataset de COORDENADAS
    * Se calculan las coordenadas cartesianas y se exporta un archivo para luego crear un dataset de locales unívocos tomando en conjunto a yelp y google
* Creación del dataset ATRIBUTOS
    * Explotando el campo "attributes" se exporta un dataset con los atributos que se consideran relevantes para el modelo de machine learning
* Dataset CHECKIN
    * Se obtiene este registro de visitas mediante el filtrado de locales tipo restaurants & bares
* Dataset USER
    * Se adecúan los tipos de datos para su posterior procesamiento

# ETL YELP - 2

## Carga de datos
=============================================================

In [5]:
# Restaurantes
dfyrst = pd.read_parquet('gs://archivos-preprocesados-henry/dfy_rest.parquet')

In [ ]:
# Tips
tip = pd.read_json('../Data/Yelp/tip.json', lines=True)

In [6]:
# Review
# review = pd.read_json('../Data/Yelp/review.json')

# Se Lee el archivo JSON de review en fragmentos
review_json_file = '../Data/Yelp/review.json'
chunksize = 1000000  # Tamaño de la muestra a ajustar para poder cargar la información.

# Se crea una lista para almacenar los datos filtrados
filtered_data = []

# Iterar sobre los fragmentos del archivo JSON
for chunk in pd.read_json(review_json_file, lines=True, chunksize=chunksize):
    # Filtrar los datos del fragmento según la lista de business_id
    filtered_chunk = chunk[chunk['business_id'].isin(dfyrst["business_id"])]
    # Agregar los datos filtrados a la lista
    filtered_data.append(filtered_chunk)

# Concatenar todos los fragmentos filtrados en un solo DataFrame
dfyrev = pd.concat(filtered_data, ignore_index=True)

## Preprocesamiento
=========================================================================

### Dataset RESTAURANTES

In [18]:
dfyrst.sample()

,business_id,name_x,city_x,postal_code,latitude_x,longitude_x,stars,review_count,is_open,state_x,...,address_full,gmap_id,latitude_y,longitude_y,avg_rating,num_of_reviews,price,address,city_y,state_y
558331,zQSEPYY7YskqIvg-9cjNdQ,Uncle Giuseppe's,Garden City,83714,43.667151,-116.278957,4.5,184,1,ID,...,"Tacos Mobile Primo, 3710 W Chinden Blvd, Garde...",0x54aef95a97a844ff:0xef2e5af9393f1631,43.625518,-116.242759,4.7,38.0,1.4,3710 W Chinden Blvd,Garden City,ID


### Dataset TIP

In [ ]:
tip.sample(2)

In [ ]:
# Selección de campos
dfytip = tip

dfytip['year'] = dfytip['date'].dt.year
dfytip['month'] = dfytip['date'].dt.month
dfytip['year_month'] = dfytip['year'].astype(str).str.slice(-2) + dfytip['month'].astype(str).str.zfill(2)

# Filtrado por restaurantes
dfytip = dfytip[dfytip['business_id'].isin(dfyrst['business_id'])]
dfytip

dfytip.info()
dfytip.sample(5)

In [ ]:
# Análisis de sentimientos a partir del campo "text"
dfytip['text'] = dfytip['text'].astype(str)

analyzer = SentimentIntensityAnalyzer()
dfytip['polarity'] = dfytip['text'].apply(lambda text: analyzer.polarity_scores(text)['compound'])
dfytip['sentiment'] = pd.cut(dfytip['polarity'], bins=[-float('inf'), -0.001, 0.0, float('inf')], labels=[-1, 0, 1])

dfytip.info()
dfytip.sample(5)

In [ ]:
# Subida del dataset a GCS
destino = 'dfytip.parquet'
dataframe_to_parquet_and_upload(dfytip,bucket_name,destino)

# dfytip.to_parquet('dfy_tips.parquet', index=False)

### Dataset REVIEW

In [7]:
# Adecuación de campos
dfyrev['date'] = pd.to_datetime(dfyrev['date'], errors='coerce')
dfyrev.reset_index(drop=True, inplace=True)

# Eliminación de duplicados: No tiene duplicados, se ha analizado fuera de este archivo

# Eliminación de nulos
dfyrev = dfyrev.dropna()

dfyrev['year'] = dfyrev['date'].dt.year
dfyrev['month'] = dfyrev['date'].dt.month
dfyrev['year_month'] = dfyrev['year'].astype(str).str.slice(-2) + dfyrev['month'].astype(str).str.zfill(2)

dfyrev = dfyrev[(dfyrev['year'] >= 2010) & (dfyrev['year'] <= 2021)]

dfyrev.info()
dfyrev.sample(2)

<class 'pandas.core.frame.DataFrame'>
Index: 4797709 entries, 0 to 4926790
Data columns (total 12 columns):
 #   Column       Dtype         
---  ------       -----         
 0   review_id    object        
 1   user_id      object        
 2   business_id  object        
 3   stars        int64         
 4   useful       int64         
 5   funny        int64         
 6   cool         int64         
 7   text         object        
 8   date         datetime64[ns]
 9   year         int32         
 10  month        int32         
 11  year_month   object        
dtypes: datetime64[ns](1), int32(2), int64(4), object(5)
memory usage: 439.2+ MB


,review_id,user_id,business_id,stars,useful,funny,cool,text,date,year,month,year_month
1183074,SKJi9ieCsGSBk8f4JhZk2w,DURXhkMrCn2ZsqP6pHwKRg,nCT64ZImq3KCuUatn8jHUw,2,1,0,1,I realize Logan's is a chain so great foods is...,2013-07-28 03:02:46,2013,7,1307
2985154,fTtQoXRf44b-MZCdcDaqWA,aAAcxzkXkSYAcmBD7_uMbA,fU02Peqq03RGER0QNW3uMA,4,0,0,0,My wife and I had the fish of the day and it w...,2016-04-01 02:10:32,2016,4,1604


In [8]:
# Se selecciona una muestra aleatoria de los datos.
n_rows = 1000000 # se toma un millón de registros.
dfyrev_sample = dfyrev.sample(n=n_rows, random_state=42)


In [9]:
# Función para calcular el puntaje de sentimiento
def compute_sentiment(text):
    analyzer = SentimentIntensityAnalyzer()
    polarity = analyzer.polarity_scores(text)['compound']
    return polarity

from concurrent.futures import ProcessPoolExecutor

In [11]:
analyzer = SentimentIntensityAnalyzer()

dfyrev_sample['polarity'] = [analyzer.polarity_scores(text)['compound'] for text in dfyrev_sample['text']]
dfyrev_sample['sentiment'] = pd.cut(dfyrev_sample['polarity'], bins=[-float('inf'), -0.001, 0.0, float('inf')], labels=[-1, 0, 1])

print(dfyrev_sample.info())
dfyrev_sample.sample(2)

<class 'pandas.core.frame.DataFrame'>
Index: 1000000 entries, 1299470 to 1250568
Data columns (total 14 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   review_id    1000000 non-null  object        
 1   user_id      1000000 non-null  object        
 2   business_id  1000000 non-null  object        
 3   stars        1000000 non-null  int64         
 4   useful       1000000 non-null  int64         
 5   funny        1000000 non-null  int64         
 6   cool         1000000 non-null  int64         
 7   text         1000000 non-null  object        
 8   date         1000000 non-null  datetime64[ns]
 9   year         1000000 non-null  int32         
 10  month        1000000 non-null  int32         
 11  year_month   1000000 non-null  object        
 12  polarity     1000000 non-null  float64       
 13  sentiment    1000000 non-null  category      
dtypes: category(1), datetime64[ns](1), float64(1), int32(2), int64(4)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,year,month,year_month,polarity,sentiment
1332654,X1eQgYxe2eiQ3hmRJ-NkYw,oWI2rSsR1YruJX8Mu13AKA,7wV_V6snbNZPRyQz5FADfg,1,9,0,0,I wish no stars is an option. I guess the star...,2017-11-17 01:50:53,2017,11,1711,-0.3370,-1
4456257,oQFWVSdCbH5iebj8Z0ftRg,wH_ObWlWi2cWdR3mUgYS8Q,jkGQQ4_LgJx3hwPtCFkzbQ,4,1,2,1,"I have been to the NYC and Philly location, Ph...",2011-12-03 18:59:20,2011,12,1112,0.9616,1


In [12]:
dfyrev_sample.drop(columns=["useful","funny","cool","text"],inplace=True)

In [13]:
dfyrev_sample.sample()

,review_id,user_id,business_id,stars,date,year,month,year_month,polarity,sentiment
3565198,RC3AJBCN6eKUfXEPbr0oOg,TvZ6zamTL18AVW56Z3nSQA,7GUjQC6k8RcGwySfJcAW_g,4,2014-09-15 22:13:45,2014,9,1409,0.9702,1


In [14]:
# Se sube a la nube.
destino = 'dfyrev.parquet'
dataframe_to_parquet_and_upload(dfyrev_sample,bucket_name,destino)

Archivo temp.parquet subido a dfyrev.parquet.


## Conclusiones
==========================================================================================================================================00

* El dataset RESTAURANTES se encuentra ya procesado y se utilizará para filtrar los locales deseados utilizando el campo 'site_id"
* Dataset TIP
    * Contiene información de reseñas con un texto un poco más corto pero con sugerencias más rápidas
    * Se adecúan los tipos de datos, no se observan valores nulos ni duplicados
    * Se aplica un análisis de sentimientos utilizando "vader"
* Dataset REVIEWS
    * Se adecúan los tipos de datos, no se observan valores nulos, se eliminan los pocos duplicados que tiene
    * Se aplica un análisis de sentimientos utilizando "vader" tomando una muestra de 1 millón de registros (contiene aprox 6 millones)

# ETL Unicos

## Contenido
El presente archivo se realiza con la intención de buscar sites repetidos en los datasets, con el objetivo de crear un listado de locales únicos para luego unir ambos datasets.

El plan es el siguiente: se conservarán todos los locales del dataset de GOOGLE, y se eliminarán los locales que se encuentren repetidos en el dataset de YELP.

## Carga de datos
===========================================================================================================================000

In [40]:
# Lectura de dataset de restaurants de GOOGLE
dfgrst_coord = pd.read_parquet('gs://archivos-preprocesados-henry/dfg_rest_coord.parquet')
dfgrst_coord.sample()

,business_id,latitude,longitude,name,state,city,postal_code,source,x,y
1115376,0x8804ef5d0d75e2d7:0x30600a4a05eb54df,43.31901,-87.951541,Downtown Pizza,WI,Grafton,53024,google,165.683686,-4632.228153


In [41]:
dfgrst_coord['x'] = dfgrst_coord['x'].round(2)
dfgrst_coord['y'] = dfgrst_coord['y'].round(2)

dfgrst_coord['latitude'] = dfgrst_coord['latitude'].round(4)
dfgrst_coord['longitude'] = dfgrst_coord['longitude'].round(4)

print(dfgrst_coord.info())
dfgrst_coord.sample(2)

<class 'pandas.core.frame.DataFrame'>
Index: 223485 entries, 2 to 3025006
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   business_id  223485 non-null  object 
 1   latitude     223485 non-null  float64
 2   longitude    223485 non-null  float64
 3   name         223485 non-null  object 
 4   state        223485 non-null  object 
 5   city         223485 non-null  object 
 6   postal_code  223485 non-null  object 
 7   source       223485 non-null  object 
 8   x            223485 non-null  float64
 9   y            223485 non-null  float64
dtypes: float64(4), object(6)
memory usage: 18.8+ MB
None


,business_id,latitude,longitude,name,state,city,postal_code,source,x,y
2350129,0x52ace9b203b8587f:0x73f5cbc1d02c71ec,45.3069,-91.6466,Brass Rail Saloon,WI,Chetek,54728,google,-128.75,-4478.93
474540,0x89c25c41a63675c1:0xbd3303ce01463eea,40.6822,-73.9097,Krazy Pizza and Wings,NY,Brooklyn,11207,google,1339.02,-4642.10


In [17]:
# Lectura de dataset de restaurants de YELP
dfyrst_coord = pd.read_parquet('gs://archivos-preprocesados-henry/dfyrst_coord.parquet')
dfyrst_coord.sample()

,business_id,latitude_x,longitude_x,name_x,state_x,city_x,postal_code,source,x,y
1814710,PkzH56GHMq0t-sNes-rHdQ,39.953592,-75.173181,New York Gyro Truck,PA,Philadelphia,19103,yelp,1249.752075,-4721.172684


In [18]:
dfyrst_coord['x'] = dfyrst_coord['x'].round(2)
dfyrst_coord['y'] = dfyrst_coord['y'].round(2)
dfyrst_coord['postal_code'] = dfyrst_coord['postal_code'].astype('int64')

dfyrst_coord['latitude_x'] = dfyrst_coord['latitude_x'].round(4)
dfyrst_coord['longitude_x'] = dfyrst_coord['longitude_x'].round(4)

print(dfyrst_coord.info())
dfyrst_coord.sample(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1991330 entries, 0 to 1991329
Data columns (total 10 columns):
 #   Column       Dtype  
---  ------       -----  
 0   business_id  object 
 1   latitude_x   float64
 2   longitude_x  float64
 3   name_x       object 
 4   state_x      object 
 5   city_x       object 
 6   postal_code  int64  
 7   source       object 
 8   x            float64
 9   y            float64
dtypes: float64(4), int64(1), object(5)
memory usage: 151.9+ MB
None


,business_id,latitude_x,longitude_x,name_x,state_x,city_x,postal_code,source,x,y
1172525,c56PgVwTX9LYQ0vqGU8mTg,40.0480,-75.4391,Handel's Homemade Ice Cream,PA,Berwyn,19312,yelp,1226.13,-4720.39
1311323,90RRzppbAwVRTD7KAxNuxQ,28.0331,-82.4069,Babylon Hookah Lounge,FL,Tampa,33617,yelp,743.08,-5574.22


## Procesamiento
==============================================================================================================

In [43]:
# Convertir la columna 'postal_code' de ambos DataFrames al mismo tipo de dato (string en este caso)
dfgrst_coord['postal_code'] = dfgrst_coord['postal_code'].astype(str)
dfyrst_coord['postal_code'] = dfyrst_coord['postal_code'].astype(str)

In [44]:
# Locales repetidos
df_gy = pd.merge(dfgrst_coord, dfyrst_coord, how='inner', on=['postal_code', 'x', 'y'])
print(df_gy.info())
df_gy.sample(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 328262 entries, 0 to 328261
Data columns (total 17 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   business_id_x  328262 non-null  object 
 1   latitude       328262 non-null  float64
 2   longitude      328262 non-null  float64
 3   name           328262 non-null  object 
 4   state          328262 non-null  object 
 5   city           328262 non-null  object 
 6   postal_code    328262 non-null  object 
 7   source_x       328262 non-null  object 
 8   x              328262 non-null  float64
 9   y              328262 non-null  float64
 10  business_id_y  328262 non-null  object 
 11  latitude_x     328262 non-null  float64
 12  longitude_x    328262 non-null  float64
 13  name_x         328262 non-null  object 
 14  state_x        328262 non-null  object 
 15  city_x         328262 non-null  object 
 16  source_y       328262 non-null  object 
dtypes: float64(6), object(11)
mem

,business_id_x,latitude,longitude,name,state,city,postal_code,source_x,x,y,business_id_y,latitude_x,longitude_x,name_x,state_x,city_x,source_y
122422,0x8864686f005810b3:0x2812aabffef9b33c,36.1886,-86.7289,Porter House Bistro,TN,Nashville,37206,google,293.39,-5133.52,YpJTxfEiXfH3iC9md0zIDg,36.1885,-86.7290,Cafe Roze,TN,Nashville,yelp
275153,0x89c6c70226951401:0x4e22f740f620f3e8,39.9666,-75.2074,King Grill,PA,Philadelphia,19104,google,1246.70,-4721.02,xq28cPEi0e8UpULdkcdghA,39.9666,-75.2073,King Grill,PA,Philadelphia,yelp


In [45]:
# Locales a eliminar en dfy
dfy_a_eliminar = df_gy['business_id_y']
dfy_a_eliminar = dfy_a_eliminar.to_frame()

print(dfy_a_eliminar.info())
dfy_a_eliminar.sample(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 328262 entries, 0 to 328261
Data columns (total 1 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   business_id_y  328262 non-null  object
dtypes: object(1)
memory usage: 2.5+ MB
None


,business_id_y
309197,CVK8ENkVyoxfIHdrupIc2A
284179,0De2juaQqHNUYs-vNEj9RA


In [48]:
# Listado final de locales a conservar en dfyrst (restaurantes de YELP)
dfy_rst_final = dfyrst_coord[~dfyrst_coord['business_id'].isin(dfy_a_eliminar['business_id_y'])]

print(dfy_rst_final.info())
dfy_rst_final.sample(2)

<class 'pandas.core.frame.DataFrame'>
Index: 1727123 entries, 0 to 1991329
Data columns (total 10 columns):
 #   Column       Dtype  
---  ------       -----  
 0   business_id  object 
 1   latitude_x   float64
 2   longitude_x  float64
 3   name_x       object 
 4   state_x      object 
 5   city_x       object 
 6   postal_code  object 
 7   source       object 
 8   x            float64
 9   y            float64
dtypes: float64(4), object(6)
memory usage: 144.9+ MB
None


,business_id,latitude_x,longitude_x,name_x,state_x,city_x,postal_code,source,x,y
1279437,qDRjceFNH7Mxv9M-ZvDitg,39.9502,-75.1705,Rittenhouse Farmers' Market,PA,Philadelphia,19103,yelp,1250.03,-4721.35
271473,6-zo580McbV5LdzjKwmpkg,39.9042,-86.0842,Raw Juice,IN,Indianapolis,46250,yelp,333.75,-4875.90


In [47]:
# Listado final de locales a conservar en dfgyrst (restaurantes de YELP y GOOGLE en conjunto)
dfgy_rst_final = pd.concat([dfgrst_coord, dfy_rst_final])

print(dfgy_rst_final.info())
dfgy_rst_final.sample(2)

<class 'pandas.core.frame.DataFrame'>
Index: 1950608 entries, 2 to 1991329
Data columns (total 15 columns):
 #   Column       Dtype  
---  ------       -----  
 0   business_id  object 
 1   latitude     float64
 2   longitude    float64
 3   name         object 
 4   state        object 
 5   city         object 
 6   postal_code  object 
 7   source       object 
 8   x            float64
 9   y            float64
 10  latitude_x   float64
 11  longitude_x  float64
 12  name_x       object 
 13  state_x      object 
 14  city_x       object 
dtypes: float64(6), object(9)
memory usage: 238.1+ MB
None


,business_id,latitude,longitude,name,state,city,postal_code,source,x,y,latitude_x,longitude_x,name_x,state_x,city_x
468193,_I4a0zwTEJsvbLVqM5cS_g,NaN,NaN,NaN,NaN,NaN,19720,yelp,1222.37,-4745.40,39.7217,-75.5552,Karribbean Delight,DE,Wilmington
1947040,suj0xJ6GgpFGLEyiH_1unw,NaN,NaN,NaN,NaN,NaN,34652,yelp,710.36,-5567.09,28.2487,-82.7283,Country Pizza & Italian Grill,FL,New Port Richey


### Exportación de datos

In [49]:
# Subida del dataset a GCS
destino = 'dfgy_rst_final.parquet'
dataframe_to_parquet_and_upload(dfgy_rst_final,bucket_name,destino)

#dfgy_rst_final.to_parquet('dfgy_rest_uniques.parquet')
#dfy_rst_final.to_parquet('dfy_rest_uniques.parquet')

Archivo temp.parquet subido a dfgy_rst_final.parquet.


## Conclusiones

El objetivo de este ETL es lograr un listado único de locales, uniendo ambos datasets de yelp y google.

* Se conserva el campo "site_id" original de cada dataset, pero como cada dataset tiene un formatto diferente, se busca otra alternativa para identificar en forma unívoca a cada site
* Para hallar locales coincidentes, se utilizan los campos de coordenadas cartesianas "x" e "y", y además el campo "postal_code"
* Finalmente, se conserva el listado original de google, y se eliminan los locales repetidos en el listado de yelp